<a href="https://colab.research.google.com/github/marketakvasova/LSEC_segmentation/blob/main/automatic_image_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Automatic segmentation of electron microscope images**

This notebook is intended for training a neural network for the task of binary segmentation of fenestrations of Liver sinusoidal entdothelial cells (LSECS).

# How to use this notebook

To train a network, first connect to a GPU (**Runtime -> Change runtime time -> Hardware accelerator -> GPU**).

If you are using a pretrained network for inference and not training, being connected only to a **CPU** is slower, but possible.

This notebook works with data saved on your Google Drive. Network training requires pairs of images and their corresponding masks saved in two diferent folders. The image-mask pairs don't need to be named exactly the same, but they should correspond when sorted alphabetically.

In [1]:
# @title  { display-mode: "form" }
#@markdown ##**Run this cell to connect to Google Drive**
#@markdown A new window will open where you will be able to connect.

#@markdown When you are connected, you can see your Drive content in the left sidebar under **Files**.

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# **1. Setup**

In [2]:
!python --version
!pip install wandb
!pip install torchmetrics
!pip install segmentation-models-pytorch

import segmentation_models_pytorch as smp
from torchmetrics.classification import Dice, BinaryJaccardIndex
import os
from google.colab import drive
import torch.cuda
from torch.utils.data import Dataset
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader, Subset
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torchvision import transforms
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.optim as optim
from torchsummary import summary
import shutil
import cv2 as cv
from numpy.lib.stride_tricks import as_strided
import pywt
from scipy.stats import norm
from google.colab.patches import cv2_imshow
import gc
import wandb
from numba import njit
from scipy.signal import convolve2d
import math

# gc.collect()
drive.mount('/content/gdrive')
model_folder = "./gdrive/MyDrive/ROI_patches/my_model"
os.makedirs(model_folder, exist_ok=True)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu" # TODO: do not even try this, if the gpu is not connected
print(DEVICE)
biomodel_folder = os.path.join(model_folder, "bioimageio_model")
biomodel_path = os.path.join(biomodel_folder, "weights.pt")
os.makedirs(biomodel_folder, exist_ok=True)
LOAD_TRAINED_MODEL = False
model_path = os.path.join(model_folder,"my_checkpoint.pth")

Python 3.10.12
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.1/281.1 kB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 12.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-man

# Utils

## Data utils

In [3]:
class MyDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = sorted([f for f in os.listdir(self.image_dir) if os.path.isfile(os.path.join(self.image_dir, f))])
        self.masks = sorted([f for f in os.listdir(self.mask_dir) if os.path.isfile(os.path.join(self.mask_dir, f))])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_path = os.path.join(self.image_dir, self.images[index])
        mask_path = os.path.join(self.mask_dir, self.masks[index]) # mask and image need to be called the same
        image = cv.imread(img_path, cv.IMREAD_GRAYSCALE).astype(np.float32)
        mask = cv.imread(mask_path, cv.IMREAD_GRAYSCALE).astype(np.float32)
        # mask /= 255
        mask[mask == 255.0] = 1

        augmentations = self.transform(image=image, mask=mask)
        image = augmentations["image"]
        mask = augmentations["mask"]

        return image, mask

def normalize_hist(img):
    clahe = cv.createCLAHE(10, tileGridSize=(11, 11))
    img = clahe.apply(img)
    img = cv.medianBlur(img, 3)
    return img


def get_loaders(img_train, mask_train, img_val, mask_val, batch_size, num_workers=0, pin_memory=True):
    train_transform, val_transform = get_transforms()

    train_data = MyDataset(
        image_dir=img_train,
        mask_dir=mask_train,
        transform=train_transform
    )
    val_data = MyDataset(
        image_dir=img_val,
        mask_dir=mask_val,
        transform=val_transform
    )

    train_loader = DataLoader(
        train_data,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=True
    )

    val_loader = DataLoader(
        val_data,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=False
    )

    return train_loader, val_loader


def get_transforms():
    train_transform = A.Compose(
        [
            # A.Rotate(limit=35, p=1.0),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.Affine(scale=(0.9, 1.1)),
            A.Normalize(
                mean = 0.5,
                std = 0.5,
                max_pixel_value=255.0,
            ),
            ToTensorV2()
        ]
    )

    val_transform = A.Compose(
        [
            A.Normalize(
                mean = 0.5,
                std = 0.5,
                max_pixel_value=255.0,
            ),
            ToTensorV2()
        ]
    )
    return train_transform, val_transform

test_transform = A.Compose(
    [
        A.Normalize(
        mean = 0.5,
        std = 0.5,
        max_pixel_value=255.0,
        ),
            ToTensorV2()
    ]
)


def merge_images(image, mask):
    merge = np.zeros((mask.shape[0], mask.shape[1], 3))
    merge[:, :, 0] = image # B channel (0, 1, 2) = (B, G, R)
    merge[:, :, 2] = image # R channel
    merge[:, :, 1] = mask # G channel
    merge[:, :, 2][mask == 255.0] = 255 # R channel
    merge = merge.astype('uint8')
    return merge


def merge_original_mask(image_path, mask_path, output_folder):
    image = cv.imread(image_path, cv.IMREAD_GRAYSCALE)
    mask = cv.imread(mask_path, cv.IMREAD_GRAYSCALE)
    merge = merge_images(image, mask)
    filename_ext = os.path.basename(image_path)
    filename, ext = os.path.splitext(filename_ext)
    cv.imwrite(os.path.join(output_folder, filename+"_original_mask_merge"+ext), merge)


def merge_masks(mask1_path, mask2_path, output_folder):
    print('merging masks')
    mask1 = cv.imread(mask1_path, cv.IMREAD_GRAYSCALE)
    mask2 = cv.imread(mask2_path, cv.IMREAD_GRAYSCALE)
    # merge = merge_images(image, mask)
    merge = np.zeros((mask1.shape[0], mask1.shape[1], 3))

    merge[:, :, 1][mask1 == 255.0] = 255
    merge[:, :, 2][mask2 == 255.0] = 255

    filename_ext = os.path.basename(mask1_path)
    filename, ext = os.path.splitext(filename_ext)
    cv.imwrite(os.path.join(output_folder, filename+"_mask_compare"+ext), merge)


def create_weighting_patches(patch_size, edge_size):
    patch = np.ones((patch_size, patch_size), dtype=float)

    # Calculate the linear decrease values
    decrease_values = np.linspace(1, 0, num=edge_size)
    decrease_values = np.tile(decrease_values, (patch_size, 1))
    increase_values = np.linspace(0, 1, num=edge_size)
    increase_values = np.tile(increase_values, (patch_size, 1))

    # Middle patch
    # Apply linear decrease to all four edges
    middle = patch.copy()
    middle[:, 0:edge_size] *= increase_values
    middle[:, patch_size-edge_size:patch_size] *= decrease_values
    middle[0:edge_size, :] *= increase_values.T
    middle[patch_size-edge_size:patch_size, :] *= decrease_values.T
    # cv2_imshow((middle*255).astype(np.uint8))

    # Left
    left = patch.copy()
    left[:, patch_size-edge_size:patch_size] *= decrease_values
    left[0:edge_size, :] *= increase_values.T
    left[patch_size-edge_size:patch_size, :] *= decrease_values.T
    # cv2_imshow((left*255).astype(np.uint8))

    # Right
    right = patch.copy()
    right[:, 0:edge_size] *= increase_values
    right[0:edge_size, :] *= increase_values.T
    right[patch_size-edge_size:patch_size, :] *= decrease_values.T
    # cv2_imshow((right*255).astype(np.uint8))

    # Top
    top = patch.copy()
    top[:, 0:edge_size] *= increase_values
    top[:, patch_size-edge_size:patch_size] *= decrease_values
    top[patch_size-edge_size:patch_size, :] *= decrease_values.T
    # cv2_imshow((top*255).astype(np.uint8))

    # Bottom
    bottom = patch.copy()
    bottom[:, 0:edge_size] *= increase_values
    bottom[:, patch_size-edge_size:patch_size] *= decrease_values
    bottom[0:edge_size, :] *= increase_values.T
    # cv2_imshow((bottom*255).astype(np.uint8))

    # Left Top edge
    top_left = patch.copy()
    top_left[:, patch_size-edge_size:patch_size] *= decrease_values
    top_left[patch_size-edge_size:patch_size, :] *= decrease_values.T
    # cv2_imshow((top_left*255).astype(np.uint8))

    # Right top edge
    top_right = patch.copy()
    top_right[:, 0:edge_size] *= increase_values
    top_right[patch_size-edge_size:patch_size, :] *= decrease_values.T
    # cv2_imshow((top_right*255).astype(np.uint8))

    # Left bottom edge
    bottom_left = patch.copy()
    bottom_left[:, patch_size-edge_size:patch_size] *= decrease_values
    bottom_left[0:edge_size, :] *= increase_values.T
    # cv2_imshow((bottom_left*255).astype(np.uint8))

    # Right Bottom edge
    bottom_right = patch.copy()
    bottom_right[:, 0:edge_size] *= increase_values
    bottom_right[0:edge_size, :] *= increase_values.T
    # cv2_imshow((bottom_right*255).astype(np.uint8))

    return middle, top_left, top, top_right, right, bottom_right, bottom, bottom_left, left


def add_mirrored_border(image, border_size, window_size):
    height, width = image.shape

    bottom_edge = window_size - ((height + border_size) % (window_size - border_size))
    right_edge = window_size - ((width + border_size) % (window_size - border_size))

    top_border = np.flipud(image[0:border_size, :])
    bottom_border = np.flipud(image[height - (border_size+bottom_edge):height, :])
    # bottom_zeros = np.zeros((bottom_edge-border_size, width), dtype = image.dtype)
    top_bottom_mirrored = np.vstack((top_border, image, bottom_border))

    left_border = np.fliplr(top_bottom_mirrored[:, 0:border_size])
    right_border = np.fliplr(top_bottom_mirrored[:, width - (border_size+right_edge):width])
    # right_zeros = np.zeros((top_bottom_mirrored.shape[0], right_edge-border_size), dtype = image.dtype)
    mirrored_image = np.hstack((left_border, top_bottom_mirrored, right_border))
    return mirrored_image

def inference_on_image_with_overlap(model, image_path):
    # window_size = 512
    # oh, ow = 50, 50
    window_size = 224
    oh, ow = 20, 20

    # out_crop =
    input_image = cv.imread(image_path, cv.IMREAD_GRAYSCALE)
    image_height, image_width = input_image.shape
    original_height, original_width = image_height, image_width

    # bottom_edge = (image_height + oh) % (window_size - oh)
    # right_edge = (image_height + ow) % (window_size - ow)

    mirrored_image = add_mirrored_border(input_image, oh, window_size)
    # print(mirrored_image.shape)
    image_height, image_width = mirrored_image.shape


    weights = np.zeros((image_height, image_width))
    # tryout = np.zeros((image_height, image_width))
    output_probs = np.zeros((image_height, image_width))
    output_mask = np.zeros((image_height, image_width))
    middle, top_left, top, top_right, right, bottom_right, bottom, bottom_left, left = create_weighting_patches(window_size, oh)

    for x in range(0, image_height-window_size+1, window_size - oh):
        for y in range(0, image_width-window_size+1, window_size - ow):
            # Choose weighting window
            # print(x, y)
            if x == 0:
                if y == 0:
                    # if original_height != window_size:
                    weighting_window = top_left
                    # print('top left')
                elif y == image_width - window_size:
                    # print('top right')
                    weighting_window = top_right
                else:
                    weighting_window = top
                    # print('top ')
            elif x == image_height - window_size:
                if y == 0:
                    weighting_window = bottom_left
                    # print('bottom left')
                elif y == image_width - window_size:
                    weighting_window = bottom_right
                    # print('bottom right')
                else:
                    weighting_window = bottom
                    # print('bottom')
            elif y == 0:
                weighting_window = left
                # print('left')
            elif y == image_width - window_size:
                weighting_window = right
                # print('right')
            else:
                weighting_window = middle
                # print('middle')
            square_section = mirrored_image[x:x + window_size, y:y + window_size]
            weights[x:x + window_size, y:y + window_size] += weighting_window
            # tryout[x:x + window_size, y:y + window_size] += np.ones((window_size, window_size))*weighting_window
            square_section = normalize_hist(square_section)
            square_tensor = test_transform(image=square_section)['image'].unsqueeze(0).to(DEVICE)  # Add batch and channel dimension

            with torch.no_grad():
                output = torch.sigmoid(model(square_tensor)).float()

            # Scale the probablity to 0-255
            output = output*255
            # output = output.to(torch.uint8)
            output_pil = output.squeeze(0).cpu().numpy().squeeze()
            # cv2_imshow(output_pil)
            output_probs[x:x+window_size, y:y+window_size] += output_pil*weighting_window
            # with torch.no_grad():
            #     output = torch.sigmoid(model(square_tensor))
            #     output = (output > 0.5).float()
            #     output = output.cpu().data.numpy().squeeze(0).squeeze()
            #     output = output*255.0
            #     # cv2_imshow(output)
            # # # Forward pass through the model
            # # with torch.no_grad():
            # #     output = torch.sigmoid(model(square_tensor)).float()

            # # Scale the probablity to 0-255

            # # output = output.to(torch.uint8)
            # # output_pil = output.squeeze(0).cpu().numpy().squeeze()
            # # cv2_imshow(output_pil)
            # output_probs[x:x+window_size, y:y+window_size] += output*weighting_window
    # Crop
    # cv.imwrite(os.path.join(output_folder, "probs"+".png"), output_probs)

    output_probs = output_probs[oh:original_height+oh, ow:original_width+ow]
    weights *= 255
    # weights = weights[:original_height, :original_width]*255
    # tryout = tryout[:original_height, :original_width]*255

    # Apply weights
    # output_probs /= weights

    # Create image from mask
    # output_mask = np.where(output_probs > 127, 255, 0)
    threshold = int(255*0.4)
    output_mask = np.where(output_probs > threshold, 255, 0)
    output_mask = output_mask.astype(np.uint8)
    return output_mask

    # filename_ext = os.path.basename(image_path)
    # filename, ext = os.path.splitext(filename_ext)
    # # cv.imwrite(os.path.join(output_folder, filename+"_mirrored"+ext), mirrored_image)

    # # Merge image with created mask
    # out_mask_path = os.path.join(output_folder, filename+"_new_mask"+ext)
    # merge = merge_images(input_image, output_mask)
    # cv.imwrite(os.path.join(output_folder, filename+"_new_mask_merge"+ext), merge)

    # # cv.imwrite(os.path.join(output_folder, filename+"_probs"+ext), output_probs)
    # cv.imwrite(out_mask_path, output_mask)
    # # cv.imwrite(os.path.join(output_folder, filename+"_weights"+ext), weights)
    # return out_mask_path

# def inference_on_image_with_overlap(model, image_path, output_folder):
#     window_size = 512
#     oh, ow = 124, 124
#     input_image = cv.imread(image_path, cv.IMREAD_GRAYSCALE)
#     image_height, image_width = input_image.shape
#     original_height, original_width = image_height, image_width
#     bottom_edge = image_height % (window_size - oh)
#     right_edge = image_width % (window_size - ow)
#     mirrored_image = np.zeros((image_height+bottom_edge, image_width+right_edge)).astype(np.uint8)
#     mirrored_image[:image_height, :image_width] = input_image
#     mirrored_image[image_height:, :image_width] = np.flipud(input_image[image_height-bottom_edge:, :])
#     mirrored_image[:, image_width:] = np.fliplr(mirrored_image[:, image_width-right_edge:image_width])
#     image_height += bottom_edge
#     image_width += right_edge
#     weights = np.zeros((image_height, image_width))
#     # tryout = np.zeros((image_height, image_width))
#     output_probs = np.zeros((image_height, image_width))
#     output_mask = np.zeros((image_height, image_width))
#     middle, top_left, top, top_right, right, bottom_right, bottom, bottom_left, left = create_weighting_patches(window_size, oh)

#     for x in range(0, image_height-window_size+1, window_size - oh):
#         for y in range(0, image_width-window_size+1, window_size - ow):
#             # Choose weighting window
#             if x == 0:
#                 if y == 0:
#                     if original_height != window_size:
#                         weighting_window = top_left
#                     else:
#                         weighting_window = np.ones((window_size, window_size))
#                 elif y == window_size - ow - 1:
#                     weighting_window = top_right
#                 else:
#                     weighting_window = top
#             elif x == window_size - oh - 1:
#                 if y == 0:
#                     weighting_window = bottom_left
#                 elif y == window_size - ow - 1:
#                     weighting_window = bottom_right
#                 else:
#                     weighting_window = bottom
#             elif y == 0:
#                 weighting_window = left
#             elif y == window_size - ow - 1:
#                 weighting_window = right
#             else:
#                 weighting_window = middle
#             square_section = mirrored_image[x:x + window_size, y:y + window_size]
#             weights[x:x + window_size, y:y + window_size] = weighting_window
#             # tryout[x:x + window_size, y:y + window_size] += np.ones((window_size, window_size))*weighting_window
#             square_section = preprocess_image(square_section)
#             square_tensor = test_transform(square_section).unsqueeze(0).to(DEVICE)  # Add batch dimension

#             # Forward pass through the model
#             with torch.no_grad():
#                 output = torch.sigmoid(model(square_tensor)).float()

#             # Scale the probablity to 0-255
#             output = output*255
#             output = output.to(torch.uint8)
#             output_pil = output.squeeze(0).cpu().numpy()
#             output_probs[x:x+window_size, y:y+window_size] += output_pil.squeeze()*weighting_window
#     # Crop
#     output_probs = output_probs[:original_height, :original_width]
#     # weights = weights[:original_height, :original_width]*255
#     # tryout = tryout[:original_height, :original_width]*255

#     # Apply weights
#     # output_probs /= weights

#     # Create image from mask
#     output_mask = np.where(output_probs > 127, 255, 0)
#     output_mask = output_mask.astype(np.uint8)
#     filename_ext = os.path.basename(image_path)
#     filename, ext = os.path.splitext(filename_ext)

#     # Merge image with created mask
#     out_mask_path = os.path.join(output_folder, filename+"_mask"+ext)
#     merge = merge_images(input_image, output_mask)
#     cv.imwrite(os.path.join(output_folder, filename+"_merge"+ext), merge)

#     cv.imwrite(os.path.join(output_folder, filename+"_probs"+ext), output_probs)
#     cv.imwrite(out_mask_path, output_mask)
#     # cv.imwrite(os.path.join(output_folder, filename+"_weights"+ext), weights)
#     return out_mask_path


def preprocess_image(image):
    image = normalize_hist(image)
    # image = nlm_filt(image)
    # image = wavelet_denoise(image, threshold=1.5)
    # image = apply_clahe(image)
    # image = cv.medianBlur(image, 5)
    return image


def apply_clahe(image):
    clahe = cv.createCLAHE(clipLimit=0.8, tileGridSize=(8, 8))
    clahe_image = clahe.apply(image)
    return clahe_image


def create_train_val_patches(train_image_folder, train_mask_folder, val_image_folder, val_mask_folder, output_folder, patch_size):
    train_image_patches_path, train_mask_patches_path = create_image_patches(train_image_folder, train_mask_folder, output_folder, patch_size, img_type='train')
    val_image_patches_path, val_mask_patches_path = create_image_patches(val_image_folder, val_mask_folder, output_folder, patch_size, img_type='val')
    return train_image_patches_path, train_mask_patches_path, val_image_patches_path, val_mask_patches_path

def create_image_patches(image_folder, mask_folder, output_folder, patch_size, img_type):
    image_patches_path = os.path.join(output_folder, img_type+'_image_patches')
    mask_patches_path = os.path.join(output_folder, img_type+'_mask_patches')
    # rejected_path = os.path.join(output_folder,'rejected')
    # print(image_path)

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    if os.path.exists(image_patches_path):
        shutil.rmtree(image_patches_path)
    os.mkdir(image_patches_path)
    if os.path.exists(mask_patches_path):
        shutil.rmtree(mask_patches_path)
    os.mkdir(mask_patches_path)
    # if os.path.exists(rejected_path):
    #     shutil.rmtree(rejected_path)
    # os.mkdir(rejected_path)

    patch_area = patch_size**2
    fenestration_area_thresh = 0.0 #0.01
    image_filenames = [f for f in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, f))]
    image_filenames = sorted(image_filenames)
    mask_filenames = [f for f in os.listdir(mask_folder) if os.path.isfile(os.path.join(mask_folder, f))]
    mask_filenames = sorted(mask_filenames)

    for image_name, mask_name in zip(image_filenames, mask_filenames):
        # if image_name.endswith(".tif"): # TODO: tohle mozna odstranit
        input_path = os.path.join(image_folder, image_name)
        mask_path = os.path.join(mask_folder, mask_name)

        img = cv.imread(input_path, cv.IMREAD_GRAYSCALE)
        mask = cv.imread(mask_path, cv.IMREAD_GRAYSCALE)
        height, width = img.shape

        shape = (height // patch_size, width // patch_size, patch_size, patch_size)
        strides = (patch_size * width , patch_size , width, 1)
        # strides = (patch_size * width , patch_size)

        # img_strided = as_strided(img, shape=(width//patch_size, height//patch_size, patch_size, patch_size),
        #              strides=img.strides + img.strides, writeable=False)
        img_strided = as_strided(img, shape=shape,
                        strides=strides, writeable=False) #TODO: check if the patches do not overlap
        mask_strided = as_strided(mask, shape=shape,
                        strides=strides, writeable=False)
        k = 0
        for i in range(img_strided.shape[0]):
            for j in range(img_strided.shape[1]):
                if k % 2 == 0:
                    img_patch = img_strided[i, j]
                    mask_patch = mask_strided[i, j]
                    # Compute the percentage of white pixels
                    fenestration_area = np.sum(mask_patch == 255)
                    # print(fenestration_area)
                    # fenestration_percentage = fenestration_area/patch_area
                    if fenestration_area >= fenestration_area_thresh:
                        patch_filename = f"{os.path.splitext(os.path.basename(image_name))[0]}_patch_{i}_{j}.tif"
                        # preprocess image
                        img_patch = preprocess_image(img_patch)
                        cv.imwrite(os.path.join(image_patches_path, patch_filename), img_patch)
                        cv.imwrite(os.path.join(mask_patches_path, patch_filename), mask_patch)
                        # print("written patch ", patch_filename)
                    else:
                        print("not writing patch")
                k += 1
    return image_patches_path, mask_patches_path


# Denoising
#   References for non-local means filtering and noise variance estimation:
#
#   [1] Antoni Buades, Bartomeu Coll, and Jean-Michel Morel, A Non-Local
#       Algorithm for Image Denoising, Computer Vision and Pattern
#       Recognition 2005. CVPR 2005, Volume 2, (2005), pp. 60-65.
#   [2] John Immerkaer, Fast Noise Variance Estimation, Computer Vision and
#       Image Understanding, Volume 64, Issue 2, (1996), pp. 300-302

def estimate_degree_of_smoothing(I): # This is how the estimation is done in Matlab (see imnlmfilt in Matlab)
    H, W = I.shape
    I = I.astype(np.float32)
    kernel = np.array([[1, -2, 1], [-2, 4, -2], [1, -2, 1]])
    conv_result = np.abs(convolve2d(I[:, :], kernel, mode='valid'))
    res = np.sum(conv_result)
    degree_of_smoothing = (res * np.sqrt(0.5 * np.pi) / (6 * (W - 2) * (H - 2)))
    if degree_of_smoothing == 0:
        degree_of_smoothing = np.finfo(np.float32).eps
    return degree_of_smoothing


def nlm_filt(image):
    window_size = 5
    search_window_size = 21
    degree_of_smoothing = estimate_degree_of_smoothing(image)
    image = cv.fastNlMeansDenoising(image, None, h = degree_of_smoothing, templateWindowSize = 5, searchWindowSize = 21)
    return image


def anscombe_transform(data):
    return 2.0 * np.sqrt(data + 3.0/8.0)


def inverse_anscombe_transform(data):
    # Reference
    # https://github.com/broxtronix/pymultiscale/blob/master/pymultiscale/anscombe.py
    return (1.0/4.0 * np.power(data, 2) +
        1.0/4.0 * np.sqrt(3.0/2.0) * np.power(data, -1.0) -
        11.0/8.0 * np.power(data, -2.0) +
        5.0/8.0 * np.sqrt(3.0/2.0) * np.power(data, -3.0) - 1.0 / 8.0)


def wavelet_denoising(data, threshold=1.5, wavelet='coif4', threshold_type='soft'):
    coeffs = pywt.wavedec2(data, wavelet = wavelet, level=3)
    coeffs[-1] = tuple(pywt.threshold(c, threshold, threshold_type) for c in coeffs[-1])
    coeffs[-2] = tuple(pywt.threshold(c, threshold, threshold_type) for c in coeffs[-2])
    coeffs[-3] = tuple(pywt.threshold(c, threshold, threshold_type) for c in coeffs[-3])
    return pywt.waverec2(coeffs, wavelet)


def wavelet_denoise(image, threshold):
    image = anscombe_transform(image)
    image = wavelet_denoising(image, threshold)
    image = inverse_anscombe_transform(image)
    # TODO: not sure this is the correct way how to do this
    image = image/np.max(image)*255
    return image.astype(np.uint8)

## Training utils

In [4]:
# This is the official implementation of BoundaryDOULoss https://arxiv.org/pdf/2308.00220.pdf
# Taken from: https://github.com/sunfan-bvb/BoundaryDoULoss/tree/main
class BoundaryDoULoss(nn.Module):
    def __init__(self, n_classes):
        super(BoundaryDoULoss, self).__init__()
        self.n_classes = n_classes

    def _one_hot_encoder(self, input_tensor):
        tensor_list = []
        for i in range(self.n_classes):
            temp_prob = input_tensor == i
            tensor_list.append(temp_prob.unsqueeze(1))
        output_tensor = torch.cat(tensor_list, dim=1)
        return output_tensor.float()

    def _adaptive_size(self, score, target):
        kernel = torch.Tensor([[0,1,0], [1,1,1], [0,1,0]])
        padding_out = torch.zeros((target.shape[0], target.shape[-2]+2, target.shape[-1]+2))
        padding_out[:, 1:-1, 1:-1] = target
        h, w = 3, 3

        Y = torch.zeros((padding_out.shape[0], padding_out.shape[1] - h + 1, padding_out.shape[2] - w + 1)).cuda()
        for i in range(Y.shape[0]):
            Y[i, :, :] = torch.conv2d(target[i].unsqueeze(0).unsqueeze(0), kernel.unsqueeze(0).unsqueeze(0).cuda(), padding=1)
        Y = Y * target
        Y[Y == 5] = 0
        C = torch.count_nonzero(Y)
        S = torch.count_nonzero(target)
        smooth = 1e-5
        alpha = 1 - (C + smooth) / (S + smooth)
        alpha = 2 * alpha - 1

        intersect = torch.sum(score * target)
        y_sum = torch.sum(target * target)
        z_sum = torch.sum(score * score)
        alpha = min(alpha, 0.8)  ## We recommend using a truncated alpha of 0.8, as using truncation gives better results on some datasets and has rarely effect on others.
        loss = (z_sum + y_sum - 2 * intersect + smooth) / (z_sum + y_sum - (1 + alpha) * intersect + smooth)

        return loss

    def forward(self, inputs, target):
        inputs = torch.softmax(inputs, dim=1)
        target = self._one_hot_encoder(target)
        target = target.squeeze(1)

        assert inputs.size() == target.size(), 'predict {} & target {} shape do not match'.format(inputs.size(), target.size())

        loss = 0.0
        for i in range(0, self.n_classes):
            loss += self._adaptive_size(inputs[:, 0], target[:, 0])#(inputs[:, i], target[:, i])
        return loss / self.n_classes


def save_checkpoint(model, model_path):#, filename="my_checkpoint.pth"):
    print("=> Saving checkpoint")
    model.save(model_path)
    # torch.save(state, filename)

def save_state_dict(model, model_path):
    print("=> Saving checkpoint")
    torch.save(model.state_dict(), model_path)

def load_state_dict(model, model_path):
    print("=> Loading checkpoint")
    model.load_state_dict(torch.load(model_path))

def validate_model(model, loader, loss_fn):
    model.eval()
    total_loss = 0.0
    total_dice_score = 0.0
    total_samples = 0
    eps = 1e-8
    with torch.no_grad():
        for idx, (x, y) in enumerate(loader):
            x = x.to(DEVICE)
            y = y.to(DEVICE).unsqueeze(1)
            # Forward
            out = model(x)
            loss = get_loss(out, y, loss_fn)
            total_loss += loss.item() * x.size(0)
            if WANDB_CONNECTED or WANDB_LOG:
                wandb.log({"val/batch loss": loss.item()})

            predicted_probs = torch.sigmoid(out)
            predicted = (predicted_probs > 0.5).float()
            intersection = torch.sum(predicted * y)
            dice_score = (2.0 * intersection + eps) / (torch.sum(predicted) + torch.sum(y) + eps)
            total_dice_score += dice_score.item() * x.size(0)

            total_samples += x.size(0)
    model.train()

    average_loss = total_loss / total_samples
    average_dice_score = total_dice_score / total_samples

    return average_loss, average_dice_score



# def validate_model(model, loader, loss_fn):
#     num_correct = 0
#     num_pixels = 0
#     dice_score = 0
#     model.eval()
#     running_loss = 0
#     losses = []
#     dice_scores = []
#     with torch.no_grad():
#         for idx, (x, y) in enumerate(loader):
#             x = x.to(DEVICE)
#             y = y.to(DEVICE).unsqueeze(1)
#             # Forward
#             preds = model(x)
#             loss = get_loss(preds, y, loss_fn)
#             # running_loss += loss.cpu()
#             losses.append(loss.cpu())

#             preds = torch.sigmoid(preds)
#             preds = (preds > 0.5).float()

#             # num_correct += (preds == y).sum()
#             # num_pixels += torch.numel(preds)
#             dice_score += (2*(preds*y).sum()) / ((preds+y).sum() + 1e-8) # this is a better predictor
#             dice_scores.append(dice_score.cpu())
#     # print(
#     #     f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f} ()"
#     # )
#     # dice_score = dice_score/(idx+1)
#     # val_loss = running_loss/(idx+1)
#     val_loss = np.mean(np.array(losses))
#     dice_score = np.mean(np.array(dice_scores))

#     # dice_score = dice_score/len(loader)
#     # val_loss = running_loss/len(loader) #TODO: not sure this is correct(dividing by batch size?)
#     # print(f"Dice score is {dice_score}")
#     # val_losses.append(running_loss/len(loader))
#     # dice_scores.append(dice_score.cpu())
#     model.train()
#     return val_loss, dice_score



# def save_predictions_as_imgs(
#         loader, model, folder="saved_images", device="cpu"
# ):
#     model.eval()
#     for idx, (x, y) in enumerate(loader):
#         x = x.to(device=device)
#         with torch.no_grad():
#             preds = torch.sigmoid(model(x))
#             preds = (preds > 0.5).float()
#         # print(f"preds max{preds.max()}")
#         # print(f"y max {y.max()}")
#         # torchvision.utils.save_image(preds, os.path.join(folder, f"pred{idx}.png"))
#         # torchvision.utils.save_image(y.unsqueeze(1), os.path.join(folder, f"pred{idx}_correct.png"))
#             imshow(preds)
#             imshow(y.unsqueeze(1))
#         break # TODO: change this so it does not loop
#     model.train()
#     print("Saving prediction as images.")

def view_prediction(loader, model, device="cpu"):
    model.eval()
    for idx, (x, y) in enumerate(loader):
        x = x.to(device=device)
        with torch.no_grad():
            output = torch.sigmoid(model(x))
            preds = (output > 0.5).float()
            preds = preds.cpu().data.numpy()
            output = output.cpu().data.numpy()
            for i in range(preds.shape[0]):
                f=plt.figure(figsize=(128,32))
                # Original image
                plt.subplot(1,5*preds.shape[0],i+1)
                x = x.cpu()
                plt.imshow(x[i, 0, :, :], cmap='gray') # preds is a batch
                plt.title('Validation image')
                # NN output(probability)
                plt.subplot(1,5*preds.shape[0],i+2)
                plt.imshow(output[i, 0, :, :], interpolation='nearest', cmap='magma') # preds is a batch
                plt.title('NN output')
                # Segmentation
                plt.subplot(1,5*preds.shape[0],i+3)
                plt.imshow(preds[i, 0, :, :], cmap='gray') # preds is a batch
                plt.title('Prediction')
                # True mask
                plt.subplot(1,5*preds.shape[0],i+4)
                plt.imshow(y.unsqueeze(1)[i, 0, :, :], cmap='gray')
                plt.title('Ground truth')
                # IoU
                plt.subplot(1,5*preds.shape[0],i+5)
                im1 = y.unsqueeze(1)[i, 0, :, :]
                im2 = preds[i, 0, :, :]
                plt.imshow(im1, alpha=0.8, cmap='Blues')
                plt.imshow(im2, alpha=0.6,cmap='Oranges')
                plt.title('IoU')

            plt.show()
            break # TODO: change this so it does not loop
    model.train()


# def getClassWeights(mask_path, train_indices):
#     mask_dir_list = sorted(os.listdir(mask_path))
#     class_count = np.zeros(2, dtype=int)
#     for i in train_indices:
#         mask = cv.imread(os.path.join(mask_path, mask_dir_list[i]), cv.IMREAD_GRAYSCALE) #np.array(Image.open(os.path.join(mask_path, mask_dir_list[i])).convert('L'), dtype=np.float32)
#         mask[mask == 255.0] = 1
#         class_count[0] += mask.shape[0]*mask.shape[1] - mask.sum()
#         class_count[1] += mask.sum()

#     n_samples = class_count.sum()
#     n_classes = 2

#     class_weights = n_samples / (n_classes * class_count)
#     return torch.from_numpy(class_weights)


## Debug

In [5]:
from matplotlib.ticker import PercentFormatter

def show_fitted_ellipses(image_path, ellipses):
    image = cv.imread(image_path)
    for ellipse in ellipses:
        if ellipse is not None:
            cv.ellipse(image, ellipse, (0, 0, 255), 1)
            center, axes, angle = ellipse
            center_x, center_y = center
            major_axis_length, minor_axis_length = axes
            rotation_angle = angle
            # print(center_x, center_y)
            cv.circle(image, (int(center_x), int(center_y)),radius=1, color=(0, 0, 255), thickness=-1)

        # print("Center:", center)
        # print("Major Axis Length:", major_axis_length)
        # print("Minor Axis Length:", minor_axis_length)
        # print("Rotation Angle:", rotation_angle)

    cv2_imshow(image)

def fit_ellipses(filtered_contours, centers):
    ellipses = []
    num_ellipses = 0
    for contour, cnt_center in zip(filtered_contours, centers):
        if len(contour) >= 5:  # Ellipse fitting requires at least 5 points
            ellipse = cv.fitEllipse(contour) # TODO: maybe try a different computation, if this does not work well on edges (probably ok)
            # ellipse = cv.minAreaRect(cnt) # the fitEllipse functions fails sometimes(when the fenestration is on the edge and only a part of it is visible)
            dist = cv.norm(cnt_center, ellipse[0])
            # print(dist)
            if dist < 20:
                ellipses.append(ellipse)
                num_ellipses += 1
            else:
                ellipses.append((None, None, None))
        else:
            ellipses.append((None, None, None))
    # print(len(filtered_contours), len(ellipses))
    return ellipses, num_ellipses

def find_fenestration_contours(image_path):
    seg_mask = cv.imread(image_path, cv.IMREAD_GRAYSCALE)
    contours, _ = cv.findContours(seg_mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    return contours
    # image = cv.cvtColor(seg_mask, cv.COLOR_GRAY2RGB)
    # image_el = image.copy()
    # cv.drawContours(image, contours, -1, (0, 0, 255), 1)
    # cv2_imshow(image)

    # Remove noise and small artifacts
    # min_contour_area = 10
    # filtered_contours = [cnt for cnt in contours if cv.contourArea(cnt) > min_contour_area]
    # return filtered_contours

def find_contour_centers(contours):
    contour_centers = []
    for cnt in contours:
        M = cv.moments(cnt)
        center_x = int(M['m10'] / (M['m00'] + 1e-10))
        center_y = int(M['m01'] / (M['m00'] + 1e-10))
        contour_centers.append((center_x, center_y))
    # print(contour_centers)
    return contour_centers

def equivalent_circle_diameter(major_axis_length, minor_axis_length):
    return math.sqrt(major_axis_length * minor_axis_length)


import seaborn as sns
import itertools



def show_statistics(fenestration_areas, fenestration_areas_from_ellipses, roundness_of_ellipses, equivalent_diameters, min_roundness=0, min_d=None, max_d=None):
    palette = itertools.cycle(sns.color_palette())
    plt.figure(figsize=(21, 5))

    # Plot histogram of fenestration areas
    plt.subplot(1, 4, 1)
    sns.histplot(fenestration_areas, stat='probability')
    # plt.hist(fenestration_areas, bins=20, color='red', edgecolor='black', density=density)
    plt.title('Histogram of Fenestration Areas')
    plt.xlabel('Area ($\mathrm{nm}^2$)')
    # plt.ylabel('Frequency')
    plt.grid(True)

    # Plot histogram of areas of fitted elipses
    plt.subplot(1, 4, 2)
    sns.histplot(fenestration_areas_from_ellipses, stat='probability', color=next(palette)) # this will be the first color (blue)
    # plt.hist(fenestration_areas_from_ellipses, bins=20, color='red', edgecolor='black', density=density)
    plt.title('Histogram of Fenestration Areas (fitted ellipses)')
    plt.xlabel('Area ($\mathrm{nm}^2$)')
    # plt.ylabel('Frequency')
    plt.grid(True)

    # Plot histogram of roundness
    plt.subplot(1, 4, 3)
    r = sns.histplot(roundness_of_ellipses, stat='probability', color=next(palette), binwidth=0.025)
    r.set(xlim=(min_roundness, None))
    # plt.hist(roundness_of_ellipses, bins=10, color='blue', edgecolor='black', density=density)
    plt.title('Histogram of Roundness')
    plt.xlabel('Roundness (-)')
    # plt.ylabel('Frequency')
    plt.grid(True)
    # print(np.array(roundness_of_ellipses).max())

    # Plot histogram of equivalent circle diameters
    plt.subplot(1, 4, 4)
    d = sns.histplot(equivalent_diameters, stat='probability', color=next(palette), binwidth=10)
    d.set(xlim=(0, max_d))
    # plt.hist(equivalent_diameters, bins=20, color='green', edgecolor='black', density=density)
    plt.title('Histogram of Equivalent Circle Diameters')
    plt.xlabel('Diameter (nm)')
    # plt.ylabel('Frequency')
    plt.grid(True)


    # plt.gca().yaxis.set_major_formatter(PercentFormatter(1))



# Mask statistics debug
# One pixel corresponds to 10.62 nm
image_path = "./gdrive/MyDrive/ROIs_manually_corrected/augment_mask/_0_379.tif"
image_path = "./gdrive/MyDrive/lsec_test/old11_CA150_NE_01_original_mask.tif" # Image from semiautomatic labeling


pixel_size_nm = 10.62
contours = find_fenestration_contours(image_path)
fenestration_areas = [cv.contourArea(cnt) * (pixel_size_nm**2) for cnt in contours]
contour_centers = find_contour_centers(contours)
ellipses, num_ellipses = fit_ellipses(contours, contour_centers)

# Show image of fitted ellipses
# show_fitted_ellipses(image_path, ellipses)

roundness_of_ellipses = []
equivalent_diameters = []
fenestration_areas_from_ellipses = []

# for ellipse in ellipses:
#     print(ellipse)
#     center, axes, angle = ellipse
#     # center_x, center_y = center
#     major_axis_length, minor_axis_length = axes
#     roundness = minor_axis_length/major_axis_length
#     roundness_of_ellipses.append(roundness)
#     # rotation_angle = angle
#     diameter = pixel_size_nm * equivalent_circle_diameter(major_axis_length, minor_axis_length)
#     equivalent_diameters.append(diameter)
#     fenestration_areas_from_ellipses.append((diameter**2)/4*math.pi)

# show_statistics(fenestration_areas, fenestration_areas_from_ellipses, roundness_of_ellipses, equivalent_diameters)


# Display the number of circles and their fitted ellipses
print("Number of fenestrations:", len(contours))
print("Number of fitted ellipses:", len(ellipses))

Number of fenestrations: 0
Number of fitted ellipses: 0


In [ ]:
# # # Wavelet filtering debug

# image_folder = "./gdrive/MyDrive/ROIs_manually_corrected/train_images"
# images = os.listdir(image_folder)
# image_name = images[0]
# image = cv.imread(os.path.join(image_folder, image_name), cv.IMREAD_GRAYSCALE)
# # cv2_imshow(image)

# denoised_image = wavelet_denoise(image)
# # cv2_imshow(denoised_image)



# U-Net definition

In [ ]:
# import torchvision.transforms.functional as TF


def double_conv(in_ch, out_ch, activation):
    if activation == 'ReLU':
        conv = nn.Sequential(
            nn.Conv2d(in_channels=in_ch,out_channels=out_ch,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=out_ch,out_channels=out_ch,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )
    elif activation == 'GeLU':
        conv = nn.Sequential(
            nn.Conv2d(in_channels=in_ch,out_channels=out_ch,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(out_ch),
            nn.GeLU(approximate='none'),
            nn.Conv2d(in_channels=out_ch,out_channels=out_ch,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(out_ch),
            nn.GeLU(approximate='none')
        )
    return conv


def padder(left_tensor, right_tensor, device: str):
  # left_tensor is the tensor on the encoder side of UNET
  # right_tensor is the tensor on the decoder side  of the UNET

    if left_tensor.shape != right_tensor.shape:
        padded = torch.zeros(left_tensor.shape)
        padded[:, :, :right_tensor.shape[2], :right_tensor.shape[3]] = right_tensor
        return padded.to(device)

    return right_tensor.to(device)


class UNET(nn.Module):
    def __init__(self, in_channels, out_channels, device, dropout_probability, activations, out_activation):
        super(UNET, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.device = device
        self.dropout = nn.Dropout(p=dropout_probability)
        self.activations = activations

        self.max_pool = nn.MaxPool2d(kernel_size=2,stride=2)

        self.down_conv_1 = double_conv(in_ch=self.in_channels,out_ch=64, activation=activations)
        self.down_conv_2 = double_conv(in_ch=64,out_ch=128, activation=activations)
        self.down_conv_3 = double_conv(in_ch=128,out_ch=256, activation=activations)
        self.down_conv_4 = double_conv(in_ch=256,out_ch=512, activation=activations)
        self.down_conv_5 = double_conv(in_ch=512,out_ch=1024, activation=activations)
        #print(self.down_conv_1)

        self.up_conv_trans_1 = nn.ConvTranspose2d(in_channels=1024,out_channels=512,kernel_size=2,stride=2)
        self.up_conv_trans_2 = nn.ConvTranspose2d(in_channels=512,out_channels=256,kernel_size=2,stride=2)
        self.up_conv_trans_3 = nn.ConvTranspose2d(in_channels=256,out_channels=128,kernel_size=2,stride=2)
        self.up_conv_trans_4 = nn.ConvTranspose2d(in_channels=128,out_channels=64,kernel_size=2,stride=2)

        self.up_conv_1 = double_conv(in_ch=1024,out_ch=512, activation=activations)
        self.up_conv_2 = double_conv(in_ch=512,out_ch=256, activation=activations)
        self.up_conv_3 = double_conv(in_ch=256,out_ch=128, activation=activations)
        self.up_conv_4 = double_conv(in_ch=128,out_ch=64, activation=activations)

        self.conv_1x1 = nn.Conv2d(in_channels=64,out_channels=self.out_channels,kernel_size=1,stride=1)
        self.out_activation = out_activation

    def forward(self, x):
        # x = x.to(self.device)
        x1 = self.down_conv_1(x)
        p1 = self.max_pool(x1)
        x2 = self.down_conv_2(p1)
        p2 = self.max_pool(x2)
        p2 = self.dropout(p2)
        x3 = self.down_conv_3(p2)
        p3 = self.max_pool(x3)
        p3 = self.dropout(p3)
        x4 = self.down_conv_4(p3)
        p4 = self.max_pool(x4)
        p4 = self.dropout(p4)
        x5 = self.down_conv_5(p4)

        # decoding
        d1 = self.up_conv_trans_1(x5)  # up transpose convolution ("up sampling" as called in UNET paper)
        pad1 = padder(x4,d1, self.device) # padding d1 to match x4 shape
        cat1 = torch.cat([x4,pad1],dim=1) # concatenating padded d1 and x4 on channel dimension(dim 1) [batch(dim 0),channel(dim 1),height(dim 2),width(dim 3)]
        cat1 = self.dropout(cat1)
        uc1 = self.up_conv_1(cat1) # 1st up double convolution

        d2 = self.up_conv_trans_2(uc1)
        pad2 = padder(x3,d2, self.device)
        cat2 = torch.cat([x3,pad2],dim=1)
        cat2 = self.dropout(cat2)
        uc2 = self.up_conv_2(cat2)

        d3 = self.up_conv_trans_3(uc2)
        pad3 = padder(x2,d3, self.device)
        cat3 = torch.cat([x2,pad3],dim=1)
        uc3 = self.up_conv_3(cat3)

        d4 = self.up_conv_trans_4(uc3)
        pad4 = padder(x1,d4, self.device)
        cat4 = torch.cat([x1,pad4],dim=1)
        uc4 = self.up_conv_4(cat4)

        conv_1x1 = self.conv_1x1(uc4)
        if self.out_activation == 'sigmoid':
            conv_1x1 = torch.sigmoid(conv_1x1)
        return conv_1x1

#**2. Create training patches**

In [ ]:
# @title  { display-mode: "form" }
#@markdown ##**Insert Google Drive paths:**

#@markdown All Google Drive paths should start with ./gdrive/MyDrive/ (Check the folder structure in the left sidebar under **Files**).

#@markdown If you want to create new 512x512 patches, check the following box. If you already have image patches, insert the folders below.


# training_images = './gdrive/MyDrive/lsecs/masks_added_small/train_images' #@param {type:"string"}
# training_masks = './gdrive/MyDrive/lsecs/masks_added_small/train_masks' #@param {type:"string"}
# validation_images = './gdrive/MyDrive/lsecs/masks_added_small/val_images' #@param {type:"string"}
# validation_masks = './gdrive/MyDrive/lsecs/masks_added_small/val_masks' #@param {type:"string"}
training_images = './gdrive/MyDrive/lsecs/cropped_cells/patches/train_image_patches' #@param {type:"string"}
training_masks = './gdrive/MyDrive/lsecs/cropped_cells/patches/train_mask_patches' #@param {type:"string"}
validation_images = './gdrive/MyDrive/lsecs/cropped_cells/patches/val_image_patches' #@param {type:"string"}
validation_masks = './gdrive/MyDrive/lsecs/cropped_cells/patches/val_mask_patches' #@param {type:"string"}

create_patches = True # @param {type:"boolean"}
output_patches_folder = './gdrive/MyDrive/lsecs/masks_added_small/patches' #@param {type:"string"}

training_images = training_images.strip()
training_masks = training_masks.strip()
validation_images = validation_images.strip()
validation_masks = validation_masks.strip()

output_patches_folder = output_patches_folder.strip()

if not os.path.exists(training_images):
    print(f'{training_images} does not exist.')
if not os.path.exists(training_masks):
    print(f'{training_masks} does not exist.')
if not os.path.exists(validation_images):
    print(f'{validation_images} does not exist.')
if not os.path.exists(validation_masks):
    print(f'{validation_masks} does not exist.')


./gdrive/MyDrive/lsecs/masks_added_small/train_images does not exist.
./gdrive/MyDrive/lsecs/masks_added_small/val_images does not exist.


In [ ]:
SAVE_PATCHES_TO_DISK = True
patch_size = 224

if create_patches:
    if SAVE_PATCHES_TO_DISK:
        # output_folder = "./gdrive/MyDrive/lsecs/cropped_selections/patches"
        print(f'Saving patches to {output_patches_folder}')
    else:
        output_patches_folder = os.getcwd()
    train_img_patches_path, train_mask_patches_path, val_img_patches_path, val_mask_patches_path = create_train_val_patches(training_images, training_masks, validation_images, validation_masks, output_patches_folder, patch_size)
else: # The patches will be read from disk
    train_img_patches_path = training_images
    train_mask_patches_path = training_masks
    val_img_patches_path = validation_images
    val_mask_patches_path = validation_masks
    # output_folder = "./gdrive/MyDrive/lsecs/cropped_selections/patches"
    # image_patches_path = os.path.join(output_folder, 'image_patches')
    # mask_patches_path = os.path.join(output_folder, 'mask_patches')

print(f'Training image patches are located in {train_img_patches_path}, {len(os.listdir(train_img_patches_path))} patches.')
print(f'Training mask patches are located in {train_mask_patches_path}')
print(f'Validation image patches are located in {val_img_patches_path}, {len(os.listdir(val_img_patches_path))} patches.')
print(f'Validation mask patches are located in {val_mask_patches_path}')

# print(len(os.listdir(train_img_patches_path)))
# print(len(os.listdir(val_img_patches_path)))

Saving patches to ./gdrive/MyDrive/lsecs/masks_added_small/patches
Training image patches are located in ./gdrive/MyDrive/lsecs/masks_added_small/patches/train_image_patches, 1334 patches.
Training mask patches are located in ./gdrive/MyDrive/lsecs/masks_added_small/patches/train_mask_patches
Validation image patches are located in ./gdrive/MyDrive/lsecs/masks_added_small/patches/val_image_patches, 472 patches.
Validation mask patches are located in ./gdrive/MyDrive/lsecs/masks_added_small/patches/val_mask_patches


In [ ]:
# # reduce the number of patches
# import shutil
# train_im = [f for f in sorted(os.listdir(train_img_patches_path)) if os.path.isfile(os.path.join(train_img_patches_path, f))]
# train_mask = [f for f in sorted(os.listdir(train_mask_patches_path)) if os.path.isfile(os.path.join(train_mask_patches_path, f))]
# val_im = [f for f in sorted(os.listdir(val_img_patches_path)) if os.path.isfile(os.path.join(val_img_patches_path, f))]
# val_mask = [f for f in sorted(os.listdir(val_mask_patches_path)) if os.path.isfile(os.path.join(val_mask_patches_path, f))]

# new_train_img_patches_path = train_img_patches_path + '_reduced'
# new_train_mask_patches_path = train_mask_patches_path + '_reduced'
# new_val_img_patches_path = val_img_patches_path + '_reduced'
# new_val_mask_patches_path = val_mask_patches_path + '_reduced'

# if not os.path.exists(new_train_img_patches_path):
#     os.makedirs(new_train_img_patches_path)
# if not os.path.exists(new_train_mask_patches_path):
#     os.makedirs(new_train_mask_patches_path)
# if not os.path.exists(new_val_img_patches_path):
#     os.makedirs(new_val_img_patches_path)
# if not os.path.exists(new_val_mask_patches_path):
#     os.makedirs(new_val_mask_patches_path)

# for i in range(len(train_im)):
#     if i % 4 == 0:
#         shutil.copyfile(os.path.join(train_img_patches_path, train_im[i]), os.path.join(new_train_img_patches_path, train_im[i]))
#         shutil.copyfile(os.path.join(train_mask_patches_path, train_mask[i]), os.path.join(new_train_mask_patches_path, train_mask[i]))
# for i in range(len(val_im)):
#     if i % 4 == 0:
#         shutil.copyfile(os.path.join(val_img_patches_path, val_im[i]), os.path.join(new_val_img_patches_path, val_im[i]))
#         shutil.copyfile(os.path.join(val_mask_patches_path, val_mask[i]), os.path.join(new_val_mask_patches_path, val_mask[i]))

# train_img_patches_path = new_train_img_patches_path
# train_mask_patches_path = new_train_mask_patches_path
# val_img_patches_path = new_val_img_patches_path
# val_mask_patches_path = new_val_mask_patches_path

# print(f'Training image patches are located in {train_img_patches_path}, {len(os.listdir(train_img_patches_path))} patches.')
# print(f'Training mask patches are located in {train_mask_patches_path}')
# print(f'Validation image patches are located in {val_img_patches_path}, {len(os.listdir(val_img_patches_path))} patches.')
# print(f'Validation mask patches are located in {val_mask_patches_path}')

Training image patches are located in ./gdrive/MyDrive/lsecs/cropped_selections/patches_med5_224/train_image_patches_reduced, 666 patches.
Training mask patches are located in ./gdrive/MyDrive/lsecs/cropped_selections/patches_med5_224/train_mask_patches_reduced
Validation image patches are located in ./gdrive/MyDrive/lsecs/cropped_selections/patches_med5_224/val_image_patches_reduced, 236 patches.
Validation mask patches are located in ./gdrive/MyDrive/lsecs/cropped_selections/patches_med5_224/val_mask_patches_reduced


# Wandb sweep

In [6]:
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau

def build_optimizer(model, config, beta1=None, beta2=None):
    if config.optimizer == "sgd":
        optimizer = optim.SGD(model.parameters(),
                              lr=config.learning_rate,
                              weight_decay=config.weight_decay,
                              momentum=config.momentum)
    elif config.optimizer == "adam":
        optimizer = optim.Adam(model.parameters(),
                               lr=config.learning_rate,
                            #    betas=(config.beta1, config.beta2),
                               weight_decay=config.weight_decay)
    return optimizer

# TRAIN_LOADER = train_loader
# VAL_LOADER = val_loader
def build_dataloaders(config): # TODO: check if there is a better way to do this
    train_image_patches_path = config.train_image_patches_path
    train_mask_patches_path = config.train_mask_patches_path
    val_image_patches_path = config.val_image_patches_path
    val_mask_patches_path = config.val_mask_patches_path
    # image_patches_path = os.path.join(image_patches_path, 'image_patches')
    # mask_patches_path = os.path.join(mask_patches_path, 'mask_patches')
    train_loader, val_loader = get_loaders(
        train_image_patches_path,
        train_mask_patches_path,
        val_image_patches_path,
        val_mask_patches_path,
        config.batch_size,
        num_workers=0,
        pin_memory=True
    )
    return train_loader, val_loader # this is the simplest way to do it, wandb train cannot take any arguments

class EarlyStopper():
    def __init__(self, patience):
        self.patience = patience
        # self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = float('inf')

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > self.min_validation_loss:
            self.counter += 1
            print(self.counter)
            if self.counter >= self.patience:
                return True
        return False

def train_epoch(model, train_loader, optimizer, loss_fn):
    model.train()
    total_loss = 0.0
    total_samples = 0
    running_loss = 0
    losses = []
    scaler = torch.cuda.amp.GradScaler()
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device=DEVICE)
        targets = targets.unsqueeze(1).to(device=DEVICE)

        # forward
        with torch.cuda.amp.autocast():
            predictions = model(data)
            loss = get_loss(predictions, targets, loss_fn)

        # backward
        optimizer.zero_grad() # Zero the gradients
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        # running_loss += loss.item()
        # losses.append(loss.item())
        total_loss += loss.item() * data.size(0)
        total_samples += data.size(0)

        if WANDB_CONNECTED or WANDB_LOG:
            wandb.log({"train/batch loss": loss.item()})

    # number_of_batches = batch_idx+1
    # mean_loss = np.mean(np.array(losses))
    mean_loss = total_loss / total_samples
    model.eval()
    return mean_loss
    # return running_loss/number_of_batches

def build_model(model_name):
    in_channels = 1
    out_channels = 1
    if '+' in model_name:
        name_parts = model_name.split('+')
        encoder = name_parts[-2]
        if name_parts[-1] == 'imagenet' or name_parts[-1] == 'ssl':
            weights = name_parts[-1]
        else:
            weights = None
    out_activation = None
    # print(weights)
    # print(out_activation)
    if model_name == 'plain_unet':
        model = UNET(
                in_channels=in_channels,
                out_channels=out_channels,
                device=DEVICE,
                dropout_probability=0,
                activations='ReLU',
                out_activation=out_activation).to(DEVICE)
    elif 'Unet++' in model_name:
        model = smp.UnetPlusPlus(
                encoder_name=encoder,
                encoder_weights=weights,
                in_channels=in_channels,
                classes=out_channels,
                activation=out_activation,).to(DEVICE)
    elif 'Linknet' in model_name:
        model = smp.Linknet(
                encoder_name=encoder,
                encoder_weights=weights,
                in_channels=in_channels,
                classes=out_channels,
                activation=out_activation,).to(DEVICE)
    elif 'FPN' in model_name:
        model = smp.FPN(
                encoder_name=encoder,
                encoder_weights=weights,
                in_channels=in_channels,
                classes=out_channels,
                activation=out_activation,).to(DEVICE)
    elif 'DeepLabV3' in model_name:
        model = smp.DeepLabV3(
                encoder_name=encoder,
                encoder_weights=weights,
                in_channels=in_channels,
                classes=out_channels,
                activation=out_activation,).to(DEVICE)
    else:
        model = smp.Unet(
                encoder_name=encoder,
                encoder_weights=weights,
                in_channels=in_channels,
                classes=out_channels,
                activation=out_activation,).to(DEVICE)
    return model

def get_loss(pred, target, func_name):
    loss_func = None
    if func_name == 'dice':
        loss_func = smp.losses.DiceLoss(mode='binary')
        loss = loss_func(pred, target)
    elif func_name == 'bcelog':
        loss_func = nn.BCEWithLogitsLoss()
        loss = loss_func(pred, target)
    elif func_name == 'jaccard':
        loss_func = smp.losses.JaccardLoss(mode='binary')
        loss = loss_func(pred, target)
    elif func_name == 'weighted_bce':
        loss_func = nn.BCEWithLogitsLoss(pos_weight = torch.tensor(4))
        loss = loss_func(pred, target)
    elif func_name == 'focal':
        loss_func = smp.losses.FocalLoss(mode='binary')
        loss = loss_func(pred, target)
    elif func_name == 'dice+bce':
        loss_func1 = smp.losses.DiceLoss(mode='binary')
        loss1 = loss_func1(pred, target)
        loss_func2 = nn.BCEWithLogitsLoss()
        loss2 = loss_func2(pred, target)
        loss = 0.5*loss1 + 0.5*loss2
    elif func_name == '5dice+95bce':
        loss_func1 = smp.losses.DiceLoss(mode='binary')
        loss1 = loss_func1(pred, target)
        loss_func2 = nn.BCEWithLogitsLoss()
        loss2 = loss_func2(pred, target)
        loss = 0.05*loss1 + 0.95*loss2
    elif func_name == '20dice+80bce':
        loss_func1 = smp.losses.DiceLoss(mode='binary')
        loss1 = loss_func1(pred, target)
        loss_func2 = nn.BCEWithLogitsLoss()
        loss2 = loss_func2(pred, target)
        loss = 0.2*loss1 + 0.8*loss2
    elif func_name == 'dice+focal':
        loss_func1 = smp.losses.DiceLoss(mode='binary')
        loss1 = loss_func1(pred, target)
        loss_func2 = smp.losses.FocalLoss(mode='binary')
        loss2 = loss_func2(pred, target)
        loss = 0.5*loss1 + 0.5*loss2
    elif func_name == 'tversky':
        loss_func = smp.losses.TverskyLoss(mode='binary', alpha=0.7, beta=0.3)
        loss = loss_func(pred, target)
    # elif func_name == 'hausdorff':

    return loss

def wandb_train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        config = wandb.config

        train_loader, val_loader = build_dataloaders(config)
        model = build_model(config.model_type)
        optimizer = build_optimizer(model, config)
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

        # best_dice_score = 0
        smallest_val_loss = 1000.0
        early_stopper = EarlyStopper(patience=10)
        for epoch in range(config.epochs):
            print(f'Epoch {epoch}')
            avg_loss = train_epoch(model, train_loader, optimizer, config.loss_function)#, loss_fn)
            metrics = {"train/loss": avg_loss, "train/epoch": epoch}
            val_loss, dice_score = validate_model(model, val_loader, config.loss_function)
            scheduler.step(val_loss)

            if early_stopper.early_stop(val_loss):
                print(f"early stop on epoch {epoch}")
                with open('./gdrive/MyDrive/lsecs/dice_score_test/train_log.txt', "a+") as file:
                    file.write(f'{config.model_type} early stop on epoch {epoch}\n')
                break

            if val_loss < smallest_val_loss:
                torch.save(model.state_dict(), os.path.join(config.model_path, f'{config.model_type}_{config.loss_function}_{config.image_denoising_methods}.pth'))
            smallest_val_loss = min(val_loss, smallest_val_loss)

            val_metrics = {"val/val_loss": val_loss,
                           "val/dice_score": dice_score}
            wandb.log({**metrics, **val_metrics})

class DictObject:
    def __init__(self, **entries):
        self.__dict__.update(entries)

def train(config, model_out_path):
    if WANDB_LOG:
        wandb.init(
            project="LSEC_segmentation",
            config=config)
        config = wandb.config
    else:
        config = DictObject(**config)

    train_loader, val_loader = build_dataloaders(config)
    model = build_model(config.model_type)
    optimizer = build_optimizer(model, config)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

    smallest_val_loss = 1000.0
    train_losses = []
    val_losses = []
    dice_scores = []

    early_stopper = EarlyStopper(patience=10)
    for epoch in range(config.num_epochs):
        print(f'Epoch {epoch}')
        model.train()
        train_loss = train_epoch(model, train_loader, optimizer, config.loss_function)
        train_losses.append(train_loss)
        val_loss, dice_score = validate_model(model, val_loader, config.loss_function)
        scheduler.step(val_loss)

        current_lr = scheduler.get_last_lr()[0]
        print("Current learning rate:", current_lr)

        if early_stopper.early_stop(val_loss):
            print(f"early stop on epoch {epoch}")
            with open('./gdrive/MyDrive/lsecs/dice_score_test/train_log.txt', "a+") as file:
                file.write(f'{config.model_type} early stop on epoch {epoch}\n')
            break
        if val_loss < smallest_val_loss:
            torch.save(model.state_dict(), os.path.join(config.model_path, f'{config.model_type}_{config.loss_function}_{config.image_denoising_methods}.pth'))
        smallest_val_loss = min(val_loss, smallest_val_loss)

        dice_scores.append(dice_score)
        val_losses.append(val_loss)
        print(f'Dice score: {dice_score}')
        # view_prediction(val_loader, model, device = DEVICE)
        print(train_loss, val_loss)
        if WANDB_LOG:
            wandb.log({"train/train_loss": train_loss,
                       "train/epoch": epoch,
                       "val/val_loss": val_loss,
                       "val/dice_score":dice_score,
                       })
    if WANDB_LOG:
        wandb.finish()

    return train_losses, val_losses, dice_scores

In [ ]:
# train_images = "./gdrive/MyDrive/lsecs/cropped_selections"
# train_masks = "./gdrive/MyDrive/lsecs/cropped_selections"
# val_images = "./gdrive/MyDrive/lsecs/cropped_selections"
# val_masks = "./gdrive/MyDrive/lsecs/cropped_selections"

output_folder = "./gdrive/MyDrive/lsecs/cropped_selections"

# wandb sweep config
sweep_config = {
    'method': 'grid'#'grid'#
    }
metric = {
    'name': 'val/dice_score',
    'goal': 'maximize'
    }

sweep_config['metric'] = metric

parameters_dict = {
    'optimizer': {
        # 'values': ['adam', 'sgd']
        'value': 'sgd'
        },
    'learning_rate': {
        'value': 0.04,
        # a flat distribution between min and max
        # 'distribution': 'uniform',
        # 'min': 0.001,
        # 'max': 0.01
      },
    'weight_decay': {
        # 'value': 0.0189,
        'value': 0.01
        # 'distribution': 'uniform',
        # 'min': 0.01,
        # 'max' : 0.02,
    },
    # sgd parameters
    'momentum':{
        'value': 0.07,
        # 'distribution': 'uniform',
        # 'min': 0.06,
        # 'max' : 0.09,
    },

    # 'dropout': {
    #     'value': 0.5,
    #     #   'values': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
    #     },
    'epochs': {
        'value': 200,
        },

    # Dataloader params
    'train_image_patches_path': {
        'value': train_img_patches_path
        },
    'train_mask_patches_path': {
        'value': train_mask_patches_path
        },
    'val_image_patches_path': {
        'value': val_img_patches_path
        },
    'val_mask_patches_path': {
        'value': val_mask_patches_path
        },
    'batch_size': {
        'value': 32,
        # # integers between min and max
        # # with evenly-distributed logarithms
        # 'distribution': 'q_log_uniform_values',
        # 'q': 2, # the discrete step of the distribution
        # 'min': 4,
        # 'max': 8,
      },
    # Adam parameters
    # 'beta1': {
    #     'distribution': 'uniform',
    #     'min': 0.95,
    #     'max' : 0.999,
    # },
    # 'beta2': {
    #     'distribution': 'uniform',
    #     'min': 0.95,
    #     'max' : 0.999,
    # },
        # 'fc_layer_size': {
    #     'values': [128, 256, 512]
    #     },
    'image_denoising_methods': {
        'value': '11_10_indiv',
        # 'values': ['no_denoise', 'med5']
        # 'values': ['nlm', 'med5']
        # 'values': ['clahe+median5', 'med7', 'median5', 'median5+clahe', 'wave1_5+med3', 'wave2_5', 'wave2_5+med5'],#['wavelet', 'wavelet+median', 'advanced median'] # k waveletu jeste pridat ruzne thresholdy
    },
    'loss_function':{
        # 'value': 'bcelog',
        # 'values': ['dice', 'dice+bce', 'dice+focal', 'tversky'],#['dice', 'bcelog', 'jaccard', 'weighted_bce', 'focal'],#, 'tversky', 'hausdorff']
        # 'values': ['dice', 'dice+bce', 'focal','bcelog', 'dice+focal'],
        'values': ['bcelog'],


    },
    'model_type':{
        # 'values': ['plain_unet', 'resnet34+imagenet', 'resnet50+imagenet', 'inceptionv4+imagenet', 'efficientnet-b7+imagenet', 'resnet18+swsl', 'resnet18+imagenet','vgg11+imagenet'], # not great
        # 'values': ['vgg11+imagenet', 'vgg13+imagenet', 'vgg16+imagenet', 'vgg19+imagenet',  'resnet18+ssl','resnet34+imagenet','resnet50+ssl', 'resnext50_32x4d+ssl'], # good
        # 'values': ['vgg11+imagenet','vgg13+imagenet', 'vgg16+imagenet', 'vgg19+imagenet',  'resnet18+ssl',  'resnet34+imagenet','resnet50+ssl', 'efficientnet-b7+imagenet'], # the best so far
        # 'value': 'vgg11+imagenet',
        # 'values':['vgg11+imagenet','vgg13+imagenet', 'resnet18+ssl', 'resnet34+imagenet', 'efficientnet-b7+imagenet'],
        # 'values':['vgg13+imagenet', 'resnet18+ssl', 'resnet34+imagenet', [
        'values': ['resnet34+imagenet']

    },
    'model_path':{
        'value': './gdrive/MyDrive/lsecs',
    },
}

sweep_config['parameters'] = parameters_dict
sweep_id = wandb.sweep(sweep_config, project="LSEC_segmentation")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: d6svk90u
Sweep URL: https://wandb.ai/dpd/LSEC_segmentation/sweeps/d6svk90u


In [ ]:
WANDB_CONNECTED = True
wandb.agent(sweep_id, wandb_train, count=510)

wandb: Agent Starting Run: qd872sax with config:
wandb: 	batch_size: 32
wandb: 	epochs: 200
wandb: 	image_denoising_methods: 11_10_indiv
wandb: 	learning_rate: 0.04
wandb: 	loss_function: bcelog
wandb: 	model_path: ./gdrive/MyDrive/lsecs
wandb: 	model_type: resnet34+imagenet
wandb: 	momentum: 0.07
wandb: 	optimizer: sgd
wandb: 	train_image_patches_path: ./gdrive/MyDrive/lsecs/masks_added_small/patches/train_image_patches
wandb: 	train_mask_patches_path: ./gdrive/MyDrive/lsecs/masks_added_small/patches/train_mask_patches
wandb: 	val_image_patches_path: ./gdrive/MyDrive/lsecs/masks_added_small/patches/val_image_patches
wandb: 	val_mask_patches_path: ./gdrive/MyDrive/lsecs/masks_added_small/patches/val_mask_patches
wandb: 	weight_decay: 0.01
wandb: Currently logged in as: marketakvasova1 (dpd). Use `wandb login --relogin` to force relogin


Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 231MB/s]


Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
1
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
1
Epoch 30
Epoch 31
Epoch 32
1
Epoch 33
2
Epoch 34
Epoch 35
Epoch 36
1
Epoch 37
2
Epoch 38
3
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
1
Epoch 44
Epoch 45
1
Epoch 46
Epoch 47
1
Epoch 48
Epoch 49
Epoch 50
Epoch 51
1
Epoch 52
2
Epoch 53
3
Epoch 54
4
Epoch 55
5
Epoch 56
6
Epoch 57
Epoch 58
Epoch 59
Epoch 60
1
Epoch 61
2
Epoch 62
3
Epoch 63
4
Epoch 64
5
Epoch 65
Epoch 66
1
Epoch 67
2
Epoch 68
3
Epoch 69
Epoch 70
1
Epoch 71
2
Epoch 72
3
Epoch 73
Epoch 74
1
Epoch 75
2
Epoch 76
3
Epoch 77
4
Epoch 78
Epoch 79
Epoch 80
1
Epoch 81
Epoch 82
Epoch 83
1
Epoch 84
2
Epoch 85
3
Epoch 86
4
Epoch 87
5
Epoch 88
Epoch 89
Epoch 90
1
Epoch 91
2
Epoch 92
3
Epoch 93
4
Epoch 94
5
Epoch 95
6
Epoch 96
7
Epoch 97
8
Epoch 98
9
Epoch 99
10
early stop on e

train/batch loss,█▅▃▂▂▃▂▂▂▂▂▃▂▃▁▁▂▂▂▂▁▁▂▁▁▁▂▁▂▂▁▂▁▂▂▁▁▁▁▂
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/loss,█▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/batch loss,█▅▃▃▂▃▁▄▃▂▂▂▂▂▃▂▂▂▁▁▂▂▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂▁▁▂
val/dice_score,▁▆▇█████████████████████████████████████
val/val_loss,█▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/batch loss,0.01965
train/epoch,98
train/loss,0.02393
val/batch loss,0.01276
val/dice_score,0.91417


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


# **3. Training**

In [ ]:
LOAD_MODEL = False
WANDB_CONNECTED = True
WANDB_LOG = True
# image_patches_path = "./gdrive/MyDrive/lsecs/cropped_selections" # + patches_image_denoising_methods
# mask_patches_path = "./gdrive/MyDrive/lsecs/cropped_selections"
model_path = os.path.join("./gdrive/MyDrive/lsecs", f"vgg11_dice+bce_no_pre_checkpoint.pth")
config = {
    'batch_size' : 32,
    'dropout' : 0.0,
    'optimizer' : 'adam',
    'num_epochs' : 200,
    'learning_rate' : 0.02,
    'weight_decay' : 0.01,
    'momentum' : 0.07,
    'train_image_patches_path': train_img_patches_path,
    'train_mask_patches_path': train_mask_patches_path,
    'val_image_patches_path': val_img_patches_path,
    'val_mask_patches_path': val_mask_patches_path,
    'image_denoising_methods': 'none',#'nlm'
    'loss_function': 'dice+bce',
    'model_type': 'vgg11+imagenet',
    'model_path':'./gdrive/MyDrive/lsecs',
}


In [ ]:
if LOAD_MODEL:
    model = build_model(config['model_type'], config['dropout'], config['loss_function'])
    model = UNET(in_channels=1, out_channels=1, device=DEVICE, dropout_probability=config['dropout'], activations = 'ReLU', out_activation=None).to(DEVICE)
    load_state_dict(model, model_path)
else:
    # WANDB_LOG = False
    train_losses, val_losses, dice_scores = train(config, model_path)

# Training evaluation

In [ ]:
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Over Time')
plt.legend()
plt.show()

In [ ]:
plt.plot(dice_scores, label='Dice score')
plt.xlabel('Epoch')
plt.ylabel('Dice score')
plt.title('Dice Score Over Time')
plt.legend()
plt.show()

# Inference debug

In [ ]:
# Inference on full images
test_image_path = "./gdrive/MyDrive/lsec_test/old11_CA150_NE_05.tif"
test_mask_path = "./gdrive/MyDrive/lsec_test/old11_CA150_NE_05_original_mask.tif"

output_folder = "./gdrive/MyDrive/lsec_test"
# model = UNET(in_channels=1, out_channels=1, device=DEVICE, dropout_probability=config['dropout'], activation=None).to(DEVICE)
model = build_model('vgg13+imagenet', 0.0, 'dice+bce')
model.load_state_dict(torch.load(model_path))
out_mask_path = inference_on_image_with_overlap(model, test_image_path, output_folder)
merge_original_mask(test_image_path, test_mask_path, output_folder)
merge_masks(out_mask_path, test_mask_path, output_folder)

Downloading: "https://download.pytorch.org/models/vgg13-c768596a.pth" to /root/.cache/torch/hub/checkpoints/vgg13-c768596a.pth
100%|██████████| 508M/508M [00:27<00:00, 19.3MB/s]


# **4. Inference loop**

In [ ]:
# class FunctionWrapper(nn.Module):
#   def __init__(self, model):
#     super(FunctionWrapper, self).__init__()
#     self.model = model

#     def forward(self, tensor):
#         denoised = preprocess_image(tensor)
#         return self.model(denoised)

# class SigmoidWrapper(nn.Module):
#     def __init__(self, model):
#         super(SigmoidWrapper, self).__init__()
#         self.model = model
#         self.sigmoid = nn.Sigmoid()

#     def forward(self, x):
#         x = self.model(x)
#         x = self.sigmoid(x)
#         return x

# model_path = os.path.join("./gdrive/MyDrive/lsecs", f"vgg13_dice+bce_nlm_checkpoint.pth")
# device = torch.device('cuda')
# model = build_model('vgg13+none', 0.0, 'dice+bce')
# if torch.cuda.is_available():
#     model.load_state_dict(torch.load(model_path))
# model_with_sigmoid = SigmoidWrapper(model)
# model_with_sigmoid.to(device=device)
# wrapper = FunctionWrapper(model_with_sigmoid)
# wrapper.to(device=device)
# full_model_path = os.path.join("./gdrive/MyDrive/lsecs", f"vgg13_dice+bce_nlm_sigmoid_checkpoint.pth")
# torch.save(wrapper, full_model_path)
# # wrapper = PreprocessingWrapper(denoise, model)

In [ ]:
# @title  { display-mode: "form" }
#@markdown ##**Insert Google Drive paths:**

#@markdown All Google Drive paths should start with ./gdrive/MyDrive/ (Check the folder structure in the left sidebar under **Files**).
input_images_folder = './gdrive/MyDrive/lsecs/dice_score_test/images' #@param {type:"string"}
output_mask_folder = './gdrive/MyDrive/lsecs/dice_score_test/my_masks' #@param {type:"string"}
model_path = './gdrive/MyDrive/lsecs/resnet34+imagenet_bcelog_11_10_indiv.pth' #@param {type:"string"}

input_images_folder = input_images_folder.strip()
output_mask_folder = output_mask_folder.strip()
model_path = model_path.strip()

if not os.path.exists(output_mask_folder):
    os.makedirs(output_mask_folder)
if not os.path.exists(input_images_folder):
    print(f'{input_images_folder} does not exist)')

In [ ]:
# # model_path = os.path.join("./gdrive/MyDrive/lsecs", f"vgg13+imagenet_dice+bce_nlm.pth")
# # Inference on full images
# # images_path = "./gdrive/MyDrive/lsecs/fenestration_seg/sem_images"
# # # test_mask_path = "./gdrive/MyDrive/lsec_test/old11_CA150_NE_05_original_mask.tif"
# # output_folder = "./gdrive/MyDrive/lsecs/fenestration_seg/new_masks"


# # images_path = "./gdrive/MyDrive/lsecs/fenestration_seg/patches/sem_images"
# masks_path = "./gdrive/MyDrive/lsecs/fenestration_seg/zapotoczny_test/edited_masks"
# masks_path = "./gdrive/MyDrive/lsecs/fenestration_seg/whole_cells/fen_mask"

# # output_folder = "./gdrive/MyDrive/lsecs/fenestration_seg/patches/new_masks"
# # if not os.path.exists(output_folder):
# #     os.makedirs(output_folder)

# image_names = [f for f in sorted(os.listdir(input_images_folder)) if os.path.isfile(os.path.join(input_images_folder, f))]
# mask_names = [f for f in sorted(os.listdir(masks_path)) if os.path.isfile(os.path.join(masks_path, f))]

# if len(image_names) != len(mask_names):
#     print(f'There are {len(image_names)} images, but {len(mask_names)} masks.')
# # print(image_names, mask_names)
# # model = build_model('vgg13+imagenet', 0.0, 'dice+bce')
# # if torch.cuda.is_available():
# #     model.load_state_dict(torch.load(model_path))
# # else:
# #     model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
# model = torch.load(model_path)

In [ ]:
# for image_name, mask_name in zip(image_names, mask_names):
image_names = [f for f in sorted(os.listdir(input_images_folder)) if os.path.isfile(os.path.join(input_images_folder, f))]
# model = torch.load_state_dict(model_path)
basename = os.path.basename(model_path).split('+')[0]
# encoder = 'vgg11'
# if 'vgg11' in basename:
#     encoder='vgg11'
encoder = basename
model = build_model(encoder+'+none')
loaded_state_dict = torch.load(model_path)
model.load_state_dict(loaded_state_dict)
model.eval()

for image_name in image_names:
    print(image_name)
    # print(f'{image_name} - {mask_name}')
    image_path = os.path.join(input_images_folder, image_name)
    # mask_path = os.path.join(masks_path, mask_name)
    filter = None
    output_mask = inference_on_image_with_overlap(model, image_path)

    filename_ext = os.path.basename(image_path)
    filename, ext = os.path.splitext(filename_ext)
    out_mask_path = os.path.join(output_mask_folder, filename+"_new_mask_resnet34"+ext)
    # Save created mask
    cv.imwrite(out_mask_path, output_mask)
    # # Merge image with created mask
    merge = merge_images(cv.imread(image_path, cv.IMREAD_GRAYSCALE), output_mask)
    cv.imwrite(os.path.join(output_mask_folder, filename+"_new_mask_merge"+ext), merge)


    # # print(out_mask_path)
    # # print(image_path, mask_path)
    # merge_original_mask(image_path, mask_path, output_mask_folder)
    # merge_masks(out_mask_path, mask_path, output_mask_folder)
    # # break

# **6. Apply cell mask**

In [ ]:
mask_path = './gdrive/MyDrive/lsecs/dice_score_test/my_masks' #@param {type:"string"}
cell_mask_path = './gdrive/MyDrive/lsecs/dice_score_test/cell_masks' #@param {type:"string"}
#@markdown If this is checked, the old masks will be deleted.
rewrite_images = False # @param {type:"boolean"}

mask_path = mask_path.strip()
cell_mask_path = cell_mask_path.strip()

images = sorted([f for f in os.listdir(mask_path) if os.path.isfile(os.path.join(mask_path, f))])
cells = sorted([f for f in os.listdir(cell_mask_path) if os.path.isfile(os.path.join(cell_mask_path, f))])

if len(images) != len(cells):
    print('The number of ground truths and created masks differs.')

for image_name, cell_name in zip(images, cells):
    print(f'Image: {image_name} - cell: {cell_name}')
    im_path = os.path.join(mask_path, image_name)
    cell_path = os.path.join(cell_mask_path, cell_name)
    image = cv.imread(im_path, cv.IMREAD_GRAYSCALE)
    cell = cv.imread(cell_path, cv.IMREAD_GRAYSCALE)
    image[cell == 0] = 0
    if rewrite_images:
        new_name = image_name
        cv.imwrite(os.path.join(mask_path, new_name), image)
    else:
        new_name = image_name
        if not os.path.exists(os.path.join(mask_path, 'single_cell')):
            os.makedirs(os.path.join(mask_path, 'single_cell'))
        print(os.path.join(mask_path, 'single_cell', new_name))
        cv.imwrite(os.path.join(mask_path, 'single_cell', new_name), image)



# **5. Exclusion of fenestrations based on diameter and roundness**

In [7]:
def remove_contour_from_mask(contour, mask):
    # Fill the contour with black pixels
    cv.drawContours(mask, [contour], -1, 0, thickness=cv.FILLED)
    return mask

def remove_fenestrations(mask, min_d, max_d, min_roundness, pixel_size_nm):
    # contours = find_fenestration_contours(mask_path)
    contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    fenestration_areas = [cv.contourArea(cnt) * (pixel_size_nm**2) for cnt in contours]
    contour_centers = find_contour_centers(contours)
    ellipses, num_ellipses = fit_ellipses(contours, contour_centers)
    # print(len(ellipses), len(contours))
    roundness_of_ellipses = []
    equivalent_diameters = []
    fenestration_areas_from_ellipses = []
    # mask = cv.imread(mask_path, cv.IMREAD_GRAYSCALE)
    # cv2_imshow(mask)
    # show_fitted_ellipses(mask_path, ellipses)

    # Remove all contours that do not fit the chosen conditions
    # # Also remove all contours that were too small to fit an ellipse
    # print(len(ellipses), len(contours))
    # print(ellipses[0])
    # print(ellipses[1])
    for contour, ellipse in zip(contours, ellipses):
        if ellipse != (None, None, None) and ellipse is not None:
            # print(ellipse)
            center, axes, _ = ellipse
            # center_x, center_y = center
            minor_axis_length, major_axis_length = axes
            # print(axes)
            if major_axis_length != 0 and major_axis_length < 20*minor_axis_length:
                roundness = minor_axis_length/major_axis_length
                if roundness >= min_roundness:
                    roundness_of_ellipses.append(roundness)
                # rotation_angle = angle
                diameter = pixel_size_nm * equivalent_circle_diameter(major_axis_length, minor_axis_length)
                # print(contour)
                # print(diameter)
                if (diameter < min_d or diameter > max_d) or  (roundness < min_roundness) or np.isnan(diameter):
                    mask = remove_contour_from_mask(contour, mask)
                else:
                    equivalent_diameters.append(diameter)
                    fenestration_areas_from_ellipses.append((diameter**2)/4*math.pi)
            else:
                mask = remove_contour_from_mask(contour, mask)
        else:
            mask = remove_contour_from_mask(contour, mask)
    # # cv2_imshow(mask)
    # # show_statistics(fenestration_areas, fenestration_areas_from_ellipses, roundness_of_ellipses, equivalent_diameters, min_roundness, min_diameter_nm, max_diameter_nm)
    # equivalent_diameters = np.array(equivalent_diameters)
    # # print(equivalent_diameters)
    # if len(equivalent_diameters) > 0:
    #     mean = int(np.nanmean(equivalent_diameters) + 0.5) # This is how to round numbers in python...
    #     std = int(np.nanstd(equivalent_diameters) + 0.5)
    #     print(f"Mean equavalent diameter: {mean} nm, std: {std} nm ")
    return mask

In [ ]:
# @title  { display-mode: "form" }
#@markdown ##**Insert the pixel size, and min and max fenestration diameters in nanometers:**

#@markdown All fenestration with a smaller or larger diameter than the chosen range will be removed from the crated masks.
#@markdown (Use dot '.' as the decimal separator, not comma ',').

#@markdown Roundness is computed as minor axis length/major axis length of a fitted ellipse.
pixel_size_nm = 9.259 #@param {type:"number"}
#@markdown ---
filter_by_diameter = False # @param {type:"boolean"}
min_diameter_nm = 105 #@param {type:"number"}
max_diameter_nm = 500 #@param {type:"number"}
#@markdown ---
filter_by_roundness = False # @param {type:"boolean"}
min_roundness = 0.2 # @param {type:"slider", min:0, max:1, step:0.1}
# #@markdown ---
# filter_by_fenestration_area = False # @param {type:"boolean"}
# min_area_nm2 = 105 #@param {type:"number"}
# max_area_nm2 = 500 #@param {type:"number"}
#@markdown ---
mask_path = './gdrive/MyDrive/lsecs/mask_edit_test' #@param {type:"string"}
#@markdown If this is checked, the old masks will be deleted.
rewrite_images = False # @param {type:"boolean"}

mask_path = mask_path.strip()
mask_names = sorted([f for f in os.listdir(mask_path) if os.path.isfile(os.path.join(mask_path, f))])

# def remove_contour_from_mask(contour, mask):
#     # Fill the contour with black pixels
#     cv.drawContours(mask, [contour], -1, 0, thickness=cv.FILLED)
#     return mask

# def remove_fenestrations(image, min_d, max_d, min_roundness, pixel_size_nm):
#     # contours = find_fenestration_contours(mask_path)
#     contours, _ = cv.findContours(image, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
#     fenestration_areas = [cv.contourArea(cnt) * (pixel_size_nm**2) for cnt in contours]
#     contour_centers = find_contour_centers(contours)
#     ellipses = fit_ellipses(contours, contour_centers)
#     roundness_of_ellipses = []
#     equivalent_diameters = []
#     fenestration_areas_from_ellipses = []
#     mask = cv.imread(mask_path, cv.IMREAD_GRAYSCALE)
#     # cv2_imshow(mask)
#     # show_fitted_ellipses(mask_path, ellipses)

#     # Remove all contours that do not fit the chosen conditions
#     # Also remove all contours that were too small to fit an ellipse
#     for contour, ellipse in zip(contours, ellipses):
#         if ellipse is not None:
#             center, axes, angle = ellipse
#             # center_x, center_y = center
#             minor_axis_length, major_axis_length = axes
#             # print(axes)
#             if major_axis_length != 0 or major_axis_length > 20*minor_axis_length:
#                 roundness = minor_axis_length/major_axis_length
#                 if roundness >= min_roundness:
#                     roundness_of_ellipses.append(roundness)
#                 # rotation_angle = angle
#                 diameter = pixel_size_nm * equivalent_circle_diameter(major_axis_length, minor_axis_length)
#                 # print(contour)
#                 # print(diameter)
#                 if filter_by_diameter and (diameter < min_d or diameter > max_d) or filter_by_roundness and (roundness < min_roundness) or np.isnan(diameter):
#                     mask = remove_contour_from_mask(contour, mask)
#                 else:
#                     equivalent_diameters.append(diameter)
#                     fenestration_areas_from_ellipses.append((diameter**2)/4*math.pi)
#             else:
#                 mask = remove_contour_from_mask(contour, mask)
#         else:
#             mask = remove_contour_from_mask(contour, mask)
#     # cv2_imshow(mask)
#     # show_statistics(fenestration_areas, fenestration_areas_from_ellipses, roundness_of_ellipses, equivalent_diameters, min_roundness, min_diameter_nm, max_diameter_nm)
#     equivalent_diameters = np.array(equivalent_diameters)
#     # print(equivalent_diameters)
#     if len(equivalent_diameters) > 0:
#         mean = int(np.nanmean(equivalent_diameters) + 0.5) # This is how to round numbers in python...
#         std = int(np.nanstd(equivalent_diameters) + 0.5)
#         print(f"Mean equavalent diameter: {mean} nm, std: {std} nm ")
#     return mask

# def remove_fenestrations(mask_path, min_d, max_d, min_roundness, pixel_size_nm):
#     contours = find_fenestration_contours(mask_path)
#     fenestration_areas = [cv.contourArea(cnt) * (pixel_size_nm**2) for cnt in contours]
#     contour_centers = find_contour_centers(contours)
#     ellipses = fit_ellipses(contours, contour_centers)
#     roundness_of_ellipses = []
#     equivalent_diameters = []
#     fenestration_areas_from_ellipses = []
#     mask = cv.imread(mask_path, cv.IMREAD_GRAYSCALE)
#     # cv2_imshow(mask)
#     # show_fitted_ellipses(mask_path, ellipses)

#     # Remove all contours that do not fit the chosen conditions
#     # Also remove all contours that were too small to fit an ellipse
#     for contour, ellipse in zip(contours, ellipses):
#         if ellipse is not None:
#             center, axes, angle = ellipse
#             # center_x, center_y = center
#             minor_axis_length, major_axis_length = axes
#             # print(axes)
#             if major_axis_length != 0 or major_axis_length > 20*minor_axis_length:
#                 roundness = minor_axis_length/major_axis_length
#                 if roundness >= min_roundness:
#                     roundness_of_ellipses.append(roundness)
#                 # rotation_angle = angle
#                 diameter = pixel_size_nm * equivalent_circle_diameter(major_axis_length, minor_axis_length)
#                 # print(contour)
#                 # print(diameter)
#                 if filter_by_diameter and (diameter < min_d or diameter > max_d) or filter_by_roundness and (roundness < min_roundness) or np.isnan(diameter):
#                     mask = remove_contour_from_mask(contour, mask)
#                 else:
#                     equivalent_diameters.append(diameter)
#                     fenestration_areas_from_ellipses.append((diameter**2)/4*math.pi)
#             else:
#                 mask = remove_contour_from_mask(contour, mask)
#         else:
#             mask = remove_contour_from_mask(contour, mask)
#     # cv2_imshow(mask)
#     # show_statistics(fenestration_areas, fenestration_areas_from_ellipses, roundness_of_ellipses, equivalent_diameters, min_roundness, min_diameter_nm, max_diameter_nm)
#     equivalent_diameters = np.array(equivalent_diameters)
#     # print(equivalent_diameters)
#     if len(equivalent_diameters) > 0:
#         mean = int(np.nanmean(equivalent_diameters) + 0.5) # This is how to round numbers in python...
#         std = int(np.nanstd(equivalent_diameters) + 0.5)
#         print(f"Mean equavalent diameter: {mean} nm, std: {std} nm ")
#     return mask


#TODO: ukazat statistiky pro celou slozku obrazku

if not rewrite_images:
    new_mask_path = os.path.join(mask_path, 'edited_masks')
    os.makedirs(new_mask_path, exist_ok=True)
else:
    new_mask_path = mask_path
# print(new_mask_path)
for mask_name in mask_names:
    # print(mask_name)
    mask_path_full = os.path.join(mask_path, mask_name)
    # print(mask_path)
    mask = cv.imread(mask_path_full, cv.IMREAD_GRAYSCALE)
    edited_mask = remove_fenestrations(mask, min_diameter_nm, max_diameter_nm, min_roundness, pixel_size_nm)
    # print(os.path.join(new_mask_path, mask_name))
    cv.imwrite(os.path.join(new_mask_path, mask_name), edited_mask)
    # cv.imwrite(os.path.join(new_mask_path, mask_name), mask)




## Display the number of circles and their fitted ellipses
# print("Number of fenestrations:", len(contours))
# print("Number of fitted ellipses:", len(ellipses))


Mean equavalent diameter: 171 nm, std: 45 nm 
Mean equavalent diameter: 193 nm, std: 47 nm 


# **7. Inference evaluation**

In [ ]:
# @title  { display-mode: "form" }
#@markdown Insert folders with cell images and their ground truth masks for comparison:
images_path = './gdrive/MyDrive/lsecs/dice_score_test/images' #@param {type:"string"}
ground_truth_mask_folder = './gdrive/MyDrive/lsecs/dice_score_test/ground_truth_masks_1205' #@param {type:"string"}
semiauto_mask_folder = './gdrive/MyDrive/lsecs/dice_score_test/semiautomatic_masks' #@param {type:"string"}

models_path = './gdrive/MyDrive/lsecs' #@param {type:"string"}
cell_mask_path = './gdrive/MyDrive/lsecs/dice_score_test/cell_masks' #@param {type:"string"}

# filter_by_diameter = False # @param {type:"boolean"}

remove_false_fenestrations = True # @param {type:"boolean"}
pixel_size_nm = 9.28 #@param {type:"number"}
min_diameter_nm = 50 #@param {type:"number"}
max_diameter_nm = 350 #@param {type:"number"}
min_roundness = 0.4 # @param {type:"slider", min:0, max:1, step:0.1}

log_file_path = './gdrive/MyDrive/lsecs/dice_score_test/log.txt'


# images_path = './gdrive/MyDrive/lsecs/dice_score_test/semiautomatic_masks'



ground_truth_mask_folder = ground_truth_mask_folder.strip()
images_path = images_path.strip()
models_path = models_path.strip()
cell_mask_path = cell_mask_path.strip()
semiauto_mask_folder = semiauto_mask_folder.strip()


model_names = sorted([f for f in os.listdir(models_path) if os.path.isfile(os.path.join(models_path, f)) and 'pt' in f])
cells = sorted([f for f in os.listdir(cell_mask_path) if os.path.isfile(os.path.join(cell_mask_path, f))])

print(model_names)



if not os.path.exists(images_path):
    print("Images folder does not exist")
    # exit()
if not os.path.exists(ground_truth_mask_folder):
    print("Folder with ground truth masks does not exist")
    # exit()

ground_truth_images = sorted([f for f in os.listdir(ground_truth_mask_folder) if os.path.isfile(os.path.join(ground_truth_mask_folder, f))])
images = sorted([f for f in os.listdir(images_path) if os.path.isfile(os.path.join(images_path, f))])
semiauto_images = sorted([f for f in os.listdir(semiauto_mask_folder) if os.path.isfile(os.path.join(semiauto_mask_folder, f))])

if len(ground_truth_images) != len(images) or len(images) != len(semiauto_images):
    print('The number of ground truths and images differs.')
    # exit()

def compute_dice_score(image1, image2):
    eps = 1e-8
    image1[image1 == 255] = 1
    image2[image2 == 255] = 1
    intersection_sum = np.logical_and(image1, image2).sum()
    dice_score = (2*intersection_sum+eps)/(image1.sum() + image2.sum() + eps)
    return dice_score

# images_path = './gdrive/MyDrive/lsecs/dice_score_test/semiautomatic_masks'
# dice_scores = []
# with open(log_file_path, "a+") as file:
#     file.write(f'{len(images)} images\n')
#     for ground_truth_mask_name, image_name, cell_name  in zip(ground_truth_images, images, cells):
#         print(f'Compare: {ground_truth_mask_name} - {image_name} - {cell_name}')
#         file.write(f'Compare: {ground_truth_mask_name} - {image_name}\n')
#         ground_truth_mask_path = os.path.join(ground_truth_mask_folder, ground_truth_mask_name)
#         image_path = os.path.join(images_path, image_name)
#         cell_path = os.path.join(cell_mask_path, cell_name)
#         cell = cv.imread(cell_path, cv.IMREAD_GRAYSCALE)
#         ground_truth_mask = cv.imread(ground_truth_mask_path, cv.IMREAD_GRAYSCALE)
#         image_mask = cv.imread(image_path, cv.IMREAD_GRAYSCALE)
#         image_mask[cell == 0] = 0
#         current_dice_score = compute_dice_score(ground_truth_mask, image_mask)
#         print(f'Image Dice score: {round(current_dice_score*100, 1)}')
#         file.write(f'Image Dice score: {round(current_dice_score*100, 1)}\n')
#         dice_scores.append(current_dice_score)

#     dice_scores = np.array(dice_scores)
#     mean_dice = round(np.mean(dice_scores)*100, 1)
#     std_dice = round(np.std(dice_scores)*100, 1)

#     print(f'Semiautomatic Mean dice: {mean_dice} +- {std_dice}\n')
#     file.write(f'Semiautomatic Mean dice: {mean_dice} += {std_dice}\n\n')


def get_fenestrations_from_image(mask):
    contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    # fenestration_areas = [cv.contourArea(cnt) * (pixel_size_nm**2) for cnt in contours]
    contour_centers = find_contour_centers(contours)
    ellipses, num_ellipses = fit_ellipses(contours, contour_centers)
    return ellipses, num_ellipses, contours, len(contours)

def get_image_stats(contours, ellipses, pixel_size_nm, min_d=0, max_d=100000, min_roundness=0):
    # contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    fenestration_areas = [cv.contourArea(cnt) * (pixel_size_nm**2) for cnt in contours]
    # contour_centers = find_contour_centers(contours)
    # ellipses, num_ellipses = fit_ellipses(contours, contour_centers)
    roundness_of_ellipses = []
    equivalent_diameters = []
    fenestration_areas_from_ellipses = []
    # mask = cv.imread(mask_path, cv.IMREAD_GRAYSCALE)
    # cv2_imshow(mask)
    # show_fitted_ellipses(mask_path, ellipses)

    # Remove all contours that do not fit the chosen conditions
    # Also remove all contours that were too small to fit an ellipse
    for contour, ellipse in zip(contours, ellipses):
        if ellipse is not None and ellipse != (None, None, None):
            # print(ellipse)
            center, axes, _ = ellipse
            # center_x, center_y = center
            minor_axis_length, major_axis_length = axes
            if major_axis_length != 0 and major_axis_length < 20*minor_axis_length: # The fitting algorithm can fail sometimes
                roundness = minor_axis_length/major_axis_length
                if roundness >= min_roundness:
                    roundness_of_ellipses.append(roundness)
                diameter_pix = equivalent_circle_diameter(major_axis_length, minor_axis_length)
                diameter = pixel_size_nm * diameter_pix
                # print(contour)
                # print(diameter)
                if (diameter < min_d or diameter > max_d) or (roundness < min_roundness) or np.isnan(diameter):
                    # mask = remove_contour_from_mask(contour, mask)
                    continue
                else:
                    equivalent_diameters.append(diameter)
                    fenestration_areas_from_ellipses.append((diameter_pix**2)/4*math.pi)
        #     else:
        #         mask = remove_contour_from_mask(contour, mask)
        # else:
        #     mask = remove_contour_from_mask(contour, mask)
    # cv2_imshow(mask)
    # show_statistics(fenestration_areas, fenestration_areas_from_ellipses, roundness_of_ellipses, equivalent_diameters, min_roundness, min_diameter_nm, max_diameter_nm)
    # equivalent_diameters = np.array(equivalent_diameters)
    # roundness_of_ellipses = np.array(roundness_of_ellipses)
    # fenestration_areas_from_ellipses = np.array(fenestration_areas_from_ellipses)
    return equivalent_diameters, roundness_of_ellipses, fenestration_areas_from_ellipses
    # # print(equivalent_diameters)
    # if len(equivalent_diameters) > 0:
    #     mean = int(np.nanmean(equivalent_diameters) + 0.5) # This is how to round numbers in python...
    #     std = int(np.nanstd(equivalent_diameters) + 0.5)
    #     print(f"Mean equavalent diameter: {mean} nm, std: {std} nm ")
    # return len(contour), num_ellipses

import seaborn as sns
# Get stats

# pixel_size_nm = 9.259

# fig, ax = plt.subplots(len(model_names)*len(images), 2, figsize=(15, 7*len(images)))

# Flatten the ax array
# ax_flat = ax.flatten()

# with open(log_file_path, "a+") as file:
    # file.write(f'{len(images)} images\n')
# i = 0
save_plots = True
save_image_masks = False
plot_path = './gdrive/MyDrive/lsecs/plots/'

for model_name in model_names:
    all_my_diameters = []
    all_gt_diameters = []
    all_s_diameters = []

    all_my_means = []
    all_gt_means = []
    all_s_means = []

    all_my_roundness = []
    all_gt_roundness = []
    all_s_roundness = []

    num_all_my_ellipses = []
    num_all_gt_ellipses = []
    num_all_s_ellipses = []

    my_dice_scores = []
    s_dice_scores = []

    all_gt_porosities_pix = []
    all_s_porosities_pix = []
    all_my_porosities_pix = []

    all_gt_porosities_ell = []
    all_s_porosities_ell = []
    all_my_porosities_ell = []

    all_gt_frequencies = []
    all_s_frequencies = []
    all_my_frequencies = []

    # file.write(f'{model_name}\n')
    print(model_name)
    model = build_model(model_name.split('+')[0]+'+none')
    loaded_state_dict = torch.load(os.path.join(models_path, model_name)) # TODO:these models do not include sigmoid and preprocessing yet
    model.load_state_dict(loaded_state_dict)
    model.eval()
    # dice_scores = []
    # dice_scores_filt = []
    for ground_truth_mask_name, image_name, semiauto_image_name, cell_name  in zip(ground_truth_images, images, semiauto_images, cells):
        print(f'Compare: {ground_truth_mask_name} - {image_name} - {semiauto_image_name} -{cell_name}')
        # file.write(f'Compare: {ground_truth_mask_name} - {image_name}\n')
        ground_truth_mask_path = os.path.join(ground_truth_mask_folder, ground_truth_mask_name)
        image_path = os.path.join(images_path, image_name)
        semiauto_image_path = os.path.join(semiauto_mask_folder, semiauto_image_name)
        cell_path = os.path.join(cell_mask_path, cell_name)
        image = cv.imread(image_path, cv.IMREAD_GRAYSCALE)
        cell = cv.imread(cell_path, cv.IMREAD_GRAYSCALE)
        ground_truth_mask = cv.imread(ground_truth_mask_path, cv.IMREAD_GRAYSCALE)
        semiauto_mask = cv.imread(semiauto_image_path, cv.IMREAD_GRAYSCALE)

        new_mask = inference_on_image_with_overlap(model, image_path)

        new_mask[cell == 0] = 0
        semiauto_mask[cell == 0] = 0
        ground_truth_mask[cell == 0] = 0


        # I need to fill the cell nucleus to compute the area
        contours, hierarchy = cv.findContours(
            cell, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

        # Get the largest object in the image(the cell)
        empty_mask = np.zeros_like(cell)
        areas = []
        for cnt in contours:
            area = cv.contourArea(cnt)
            areas.append(area)
        areas = np.array(areas)
        max_area_idx = np.argmax(areas)

        # Fill this object and compute its area
        c = 0
        for cnt in contours:
            if c == max_area_idx:
                cv.drawContours(empty_mask, [cnt], -1, 1, thickness=cv.FILLED)
                cell_contour, hierarchy = cv.findContours(
                    empty_mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
            c += 1
        cell_area = float(np.sum(empty_mask))


        # Remove fenestrations
        new_mask_filt = remove_fenestrations(new_mask.copy(), min_diameter_nm, max_diameter_nm, min_roundness, pixel_size_nm)
        s_mask_filt = remove_fenestrations(semiauto_mask.copy(), min_diameter_nm, max_diameter_nm, min_roundness, pixel_size_nm)
        gt_mask_filt = remove_fenestrations(ground_truth_mask.copy(), min_diameter_nm, max_diameter_nm, min_roundness, pixel_size_nm)

        if save_image_masks:
            cv.imwrite(os.path.join(plot_path, image_name), new_mask_filt)

        # Save filtered masks
            merge = np.zeros((new_mask_filt.shape[0], new_mask_filt.shape[1], 3))
            merge = merge.astype('uint8')
            merge[:, :, 2][gt_mask_filt == 255.0] = 255 #R
            merge2 = merge.copy()
            merge3 = merge.copy()
            merge[:, :, 0][s_mask_filt == 255.0] = 255 #B
            merge[:, :, 1][s_mask_filt == 255.0] = 100 #G, so the blue is more visible

            merge[:, :, 1][(s_mask_filt == 255.0) & (gt_mask_filt == 255.0)] = 255 #G

            cv.imwrite(os.path.join(plot_path, image_name+"_mask_compare_semiautomatic"+'.png'), merge)

            merge2[:, :, 1][new_mask_filt == 255.0] = 255 #G
            merge2[:, :, 0][(new_mask_filt == 255.0) & (gt_mask_filt == 255.0)] = 255 #B
            cv.imwrite(os.path.join(plot_path, image_name+"_mask_compare_automatic"+'.png'), merge2)

            # merge3[:, :, 0][s_mask_filt == 255.0] = 255 #B
            # merge3[:, :, 1][s_mask_filt == 255.0] = 100 #G, so the blue is more visible
            # merge3[:, :, 1][new_mask_filt == 255.0] = 255 #G
            # cv.imwrite(os.path.join(plot_path, image_name+"_mask_compare"+'.png'), merge3)




        gt_cell_area_pix = np.sum(gt_mask_filt/255)
        s_cell_area_pix = np.sum(s_mask_filt/255)
        my_cell_area_pix = np.sum(new_mask_filt/255)

        gt_fen_area = float(gt_cell_area_pix)
        s_fen_area = float(s_cell_area_pix)
        my_fen_area = float(my_cell_area_pix)

        gt_porosity_pix = round(gt_fen_area/cell_area*100, 1)
        s_porosity_pix = round(s_fen_area/cell_area*100, 1)
        my_porosity_pix = round(my_fen_area/cell_area*100, 1)

        all_gt_porosities_pix.append(gt_porosity_pix)
        all_s_porosities_pix.append(s_porosity_pix)
        all_my_porosities_pix.append(my_porosity_pix)

        # print(f'gt: {gt_porosity_pix}, s: {s_porosity_pix}, my: {my_porosity_pix}')



        # Compute Dice scores
        my_current_dice = compute_dice_score(gt_mask_filt, new_mask_filt)
        s_current_dice = compute_dice_score(gt_mask_filt, s_mask_filt)

        my_dice_scores.append(my_current_dice)
        s_dice_scores.append(s_current_dice)
        # print(f'Image Dice score: {round(my_current_dice*100, 1)}')
        #

        # my data
        my_ellipses, num_my_ellipses, my_objects, num_all_my_objects = get_fenestrations_from_image(new_mask)
        my_equivalent_diameters, my_roundness_of_ellipses, my_fenestration_areas_from_ellipses = get_image_stats(my_objects, my_ellipses, pixel_size_nm, min_d=50, max_d=400, min_roundness=0.4)
        my_ell_area = np.sum(np.array(my_fenestration_areas_from_ellipses))
        all_my_porosities_ell.append(round(my_ell_area/cell_area*100, 1))
        # ground truth
        ellipses, num_ellipses, objects, num_all_objects = get_fenestrations_from_image(ground_truth_mask)
        equivalent_diameters, roundness_of_ellipses, fenestration_areas_from_ellipses = get_image_stats(objects, ellipses, pixel_size_nm, min_d=50, max_d=400, min_roundness=0.4)
        gt_ell_area = np.sum(np.array(fenestration_areas_from_ellipses))
        all_gt_porosities_ell.append(round(gt_ell_area/cell_area*100, 1))
        # semiautomatic_data
        s_ellipses, s_num_ellipses, s_objects, s_num_all_objects = get_fenestrations_from_image(semiauto_mask)
        s_equivalent_diameters, s_roundness_of_ellipses, s_fenestration_areas_from_ellipses = get_image_stats(s_objects, s_ellipses, pixel_size_nm, min_d=50, max_d=400, min_roundness=0.4)
        s_ell_area = np.sum(np.array(s_fenestration_areas_from_ellipses))
        all_s_porosities_ell.append(round(s_ell_area/cell_area*100, 1))

        all_my_diameters.extend(my_equivalent_diameters)
        all_gt_diameters.extend(equivalent_diameters)
        all_s_diameters.extend(s_equivalent_diameters)

        all_my_roundness.extend(my_roundness_of_ellipses)
        all_gt_roundness.extend(roundness_of_ellipses)
        all_s_roundness.extend(s_roundness_of_ellipses)


        n_my_ellipses = len(my_equivalent_diameters)
        n_s_ellipses = len(s_equivalent_diameters)
        n_gt_ellipses = len(equivalent_diameters)

        gt_freq = n_gt_ellipses/(cell_area*((pixel_size_nm/1000)**2))
        s_freq = n_s_ellipses/(cell_area*((pixel_size_nm/1000)**2))
        my_freq = n_my_ellipses/(cell_area*((pixel_size_nm/1000)**2))

        all_gt_frequencies.append(round(gt_freq, 1))
        all_s_frequencies.append(round(s_freq, 1))
        all_my_frequencies.append(round(my_freq, 1))

        num_all_my_ellipses.append(n_my_ellipses)
        num_all_gt_ellipses.append(n_gt_ellipses)
        num_all_s_ellipses.append(n_s_ellipses)

        gt_mean = round(np.mean(np.array(equivalent_diameters)))
        gt_std = round(np.std(np.array(equivalent_diameters)))
        my_mean = round(np.mean(np.array(my_equivalent_diameters)))
        my_std = round(np.std(np.array(my_equivalent_diameters)))
        s_mean = round(np.mean(np.array(s_equivalent_diameters)))
        s_std = round(np.std(np.array(s_equivalent_diameters)))


        all_my_means.append(my_mean)
        all_gt_means.append(gt_mean)
        all_s_means.append(s_mean)

        fig, ax = plt.subplots(3, 2, figsize=(15, 23))
        ax = ax.flatten()
        # plt.subplots(1, 2, figsize=(15, 7))
        sns.set_theme()  # This changes the look of plots.
        nbins = 20
        # Calculate density for each dataset
        hist_x, bin_edges_x  = np.histogram(equivalent_diameters, bins=20, range = (min_diameter_nm, max_diameter_nm), density=True)
        hist_y, bin_edges_y = np.histogram(my_equivalent_diameters, bins=20, range = (min_diameter_nm, max_diameter_nm), density=True)
        hist_z, bin_edges_z = np.histogram(s_equivalent_diameters, bins=20, range = (min_diameter_nm, max_diameter_nm), density=True)
        hist_x = hist_x*np.diff(bin_edges_x)
        hist_y = hist_y*np.diff(bin_edges_y)
        hist_z = hist_z*np.diff(bin_edges_z)

        bin_width = (max_diameter_nm - min_diameter_nm)/3/nbins
        shift1 = bin_width/2
        shift2 = bin_width/2 + bin_width
        shift3 = bin_width/2 + 2*bin_width


        ax[0].bar(bin_edges_x[:-1]+shift1, hist_x, color='r', alpha=0.9, width=bin_width)
        ax[0].bar(bin_edges_y[:-1]+shift2, hist_y, color='g', alpha=0.9, width=bin_width)
        ax[0].bar(bin_edges_z[:-1]+shift3, hist_z, color='b', alpha=0.9, width=bin_width)
        # ax[0].legend([f'Ground truth, mean: {gt_mean} nm\n{len(equivalent_diameters)} fenestrations', f'Automatic, mean: {my_mean} nm\n{len(my_equivalent_diameters)} fenestrations', f'Semiautomatic, mean: {s_mean} nm\n{len(s_equivalent_diameters)} fenestrations'], fontsize='small')
        ax[0].legend([f'Ground truth\nmean d = {gt_mean} nm', f'Automatic\nmean d = {my_mean} nm', f'Semiautomatic\nmean d = {s_mean} nm'], fontsize='medium')

        # Add title and axis labels
        # ax[0].set_title(image_name)
        ax[0].set_xlabel('Equivalent diameter (nm)', fontsize=16)
        ax[0].set_ylabel('Probability', fontsize=16)
        ax[0].set_xlim(min_diameter_nm-20, max_diameter_nm+20)

        # x_ticks = [50, 100, 150, 200, 250, 300, 350, 400]  # Define which x ticks to show
        x_ticks = list(range(min_diameter_nm, max_diameter_nm+1, 50))
        ax[0].set_xticks(x_ticks)  # Set the x ticks
        ax[0].set_xticklabels(x_ticks)  # Set the labels for the x ticks
        ax[0].tick_params(axis='x', labelsize=12)
        ax[0].tick_params(axis='y', labelsize=12)


        # Calculate density for each dataset
        nbins = 12
        hist_x, bin_edges_x  = np.histogram(roundness_of_ellipses, bins=nbins, range = (min_roundness, 1), density=True)
        hist_y, bin_edges_y = np.histogram(my_roundness_of_ellipses, bins=nbins, range = (min_roundness, 1), density=True)
        hist_z, bin_edges_z = np.histogram(s_roundness_of_ellipses, bins=nbins, range = (min_roundness, 1), density=True)
        hist_x = hist_x*np.diff(bin_edges_x)
        hist_y = hist_y*np.diff(bin_edges_y)
        hist_z = hist_z*np.diff(bin_edges_z)

        bin_width = (1 - min_roundness)/3/nbins
        shift1 = bin_width/2
        shift2 = bin_width/2 + bin_width
        shift3 = bin_width/2 + 2*bin_width

        ax[1].bar(bin_edges_x[:-1]+shift1, hist_x, color='r', alpha=0.9, width=bin_width)
        ax[1].bar(bin_edges_y[:-1]+shift2, hist_y, color='g', alpha=0.9, width=bin_width)
        ax[1].bar(bin_edges_z[:-1]+shift3, hist_z, color='b', alpha=0.9, width=bin_width)
        # # i += 1
        # plt.subplot(1, 2)
        # ax[1].hist([roundness_of_ellipses, my_roundness_of_ellipses, s_roundness_of_ellipses], color=['g','r','b'], alpha=0.8, density=True)
        ax[1].legend(['Ground truth', 'Automatic', 'Semiautomatic'], fontsize='medium')
        # Add title and axis labels
        # ax[1].set_title(image_name)
        ax[1].set_xlabel('Roundness of fitted ellipses', fontsize=16)
        ax[1].set_ylabel('Probability', fontsize=16)
        ax[1].set_xlim(min_roundness - 0.02, 1.02)
        ax[1].tick_params(axis='x', labelsize=12)
        ax[1].tick_params(axis='y', labelsize=12)

        # Show image patch
        r = slice(3600,4200)
        c = slice(2600,3200)
        image_patch = image[r, c]
        show_size = (300, 300)
        image_patch = cv.resize(image_patch, show_size)
        ax[2].imshow(image_patch, cmap='gray', vmin=0, vmax=255)  # Specify min and max values
        ax[2].set_title('Image patch example (600x600 pixels)', fontsize=16)
        ax[2].axis('off')

        ax[3].axis('off')

        gt_patch = ground_truth_mask[r, c]
        s_patch = semiauto_mask[r, c]
        my_patch = new_mask[r, c]
        # print(gt_patch.shape)

        gt_patch = remove_fenestrations(gt_patch, min_diameter_nm, max_diameter_nm, min_roundness, pixel_size_nm)
        s_patch = remove_fenestrations(s_patch, min_diameter_nm, max_diameter_nm, min_roundness, pixel_size_nm)
        my_patch = remove_fenestrations(my_patch, min_diameter_nm, max_diameter_nm, min_roundness, pixel_size_nm)

        # image_patch = cv.resize(image_patch, show_size, interpolation=cv.INTER_NEAREST)

        merge = np.zeros((gt_patch.shape[0], gt_patch.shape[1], 3))
        merge[:, :, 0][gt_patch == 255] = 255 # R channel
        merge = merge.astype('uint8')
        ax[4].imshow(merge, cmap='gray', vmin=0, vmax=255)  # Specify min and max values
        ax[4].set_title('Ground truth mask', fontsize=16)
        ax[4].axis('off')

        merge[:, :, 2][s_patch == 255] = 255 # B channel
        merge[:, :, 1][s_patch == 255] = 100 # G channel
        merge[:, :, 1][my_patch == 255] = 255 # G channel
        merge = merge.astype('uint8')

        ax[5].imshow(merge, cmap='gray', vmin=0, vmax=255)  # Specify min and max values
        ax[5].set_title('Mask comparison', fontsize=16)
        ax[5].axis('off')
        leg = f'R = automatic FN & semiautomatic FN\nG = automatic FP\nB = semiautomatic FP'
        leg += '\nC = automatic FN & semiautomatic FN'
        leg += '\nM = automatic FN & semiautomatic TP'
        leg += '\nY  = automatic TP & semiautomatic FN'
        leg += '\nW = automatic TP & semiautomatic TP'
        ax[5].text(0, merge.shape[0]+210, leg, ha='left', fontsize=16)
        if save_plots:
            plt.savefig(os.path.join(plot_path, f'stats_{image_name}.svg'), format='svg')


        plt.show()

    # Stats for whole dataset
    gt_mean = round(np.mean(np.array(all_gt_diameters)))
    gt_std = round(np.std(np.array(all_gt_diameters)))
    my_mean = round(np.mean(np.array(all_my_diameters)))
    my_std = round(np.std(np.array(all_my_diameters)))
    s_mean = round(np.mean(np.array(all_s_diameters)))
    s_std = round(np.std(np.array(all_s_diameters)))

    fig, ax = plt.subplots(1, 2, figsize=(13, 7))
    ax = ax.flatten()
    sns.set_theme()
    # Calculate density for each dataset
    nbins=20
    hist_x, bin_edges_x  = np.histogram(all_gt_diameters, bins=nbins, range = (min_diameter_nm, max_diameter_nm), density=True)
    hist_y, bin_edges_y = np.histogram(all_my_diameters, bins=nbins, range = (min_diameter_nm, max_diameter_nm), density=True)
    hist_z, bin_edges_z = np.histogram(all_s_diameters, bins=nbins, range = (min_diameter_nm, max_diameter_nm), density=True)
    hist_x = hist_x*np.diff(bin_edges_x)
    hist_y = hist_y*np.diff(bin_edges_y)
    hist_z = hist_z*np.diff(bin_edges_z)

    bin_width = (max_diameter_nm - min_diameter_nm)/3/nbins
    shift1 = bin_width/2
    shift2 = bin_width/2 + bin_width
    shift3 = bin_width/2 + 2*bin_width

    ax[0].bar(bin_edges_x[:-1]+shift1, hist_x, color='r', alpha=0.9, width=bin_width)
    ax[0].bar(bin_edges_y[:-1]+shift2, hist_y, color='g', alpha=0.9, width=bin_width)
    ax[0].bar(bin_edges_z[:-1]+shift3, hist_z, color='b', alpha=0.9, width=bin_width)
    # ax[0].legend([f'Ground truth, mean: {gt_mean} nm\n{len(all_gt_diameters)} fenestrations', f'Automatic, mean: {my_mean} nm\n{len(all_my_diameters)} fenestrations', f'Semiautomatic, mean: {s_mean} nm\n{len(all_s_diameters)} fenestrations'], fontsize='small')
    ax[0].legend([f'Ground truth\nmean d = {gt_mean} nm', f'Automatic\nmean d = {my_mean} nm', f'Semiautomatic\nmean d = {s_mean} nm'], fontsize='medium')

    # Add title and axis labels
    ax[0].set_title('Whole dataset', fontsize=22)
    ax[0].set_xlabel('Equivalent diameter (nm)', fontsize=16)
    ax[0].set_ylabel('Probability', fontsize=16)
    ax[0].set_xlim(min_diameter_nm-20, max_diameter_nm+20)

    # x_ticks = [50, 100, 150, 200, 250, 300, 350, 400]  # Define which x ticks to show
    x_ticks = list(range(min_diameter_nm, max_diameter_nm+1, 50))
    ax[0].set_xticks(x_ticks)  # Set the x ticks
    ax[0].set_xticklabels(x_ticks)  # Set the labels for the x ticks
    ax[0].tick_params(axis='x', labelsize=12)
    ax[0].tick_params(axis='y', labelsize=12)


    # Stats for whole dataset
    gt_mean = round(np.mean(np.array(all_gt_roundness)))
    gt_std = round(np.std(np.array(all_gt_roundness)))
    my_mean = round(np.mean(np.array(all_my_roundness)))
    my_std = round(np.std(np.array(all_my_roundness)))
    s_mean = round(np.mean(np.array(all_s_roundness)))
    s_std = round(np.std(np.array(all_s_roundness)))

    # Calculate density for each dataset


    nbins=12
    hist_x, bin_edges_x  = np.histogram(all_gt_roundness, bins=nbins, range = (min_roundness, 1), density=True)
    hist_y, bin_edges_y = np.histogram(all_my_roundness, bins=nbins, range = (min_roundness, 1), density=True)
    hist_z, bin_edges_z = np.histogram(all_s_roundness, bins=nbins, range = (min_roundness, 1), density=True)
    hist_x = hist_x*np.diff(bin_edges_x)
    hist_y = hist_y*np.diff(bin_edges_y)
    hist_z = hist_z*np.diff(bin_edges_z)

    bin_width = (1 - min_roundness)/3/nbins
    shift1 = bin_width/2
    shift2 = bin_width/2 + bin_width
    shift3 = bin_width/2 + 2*bin_width

    ax[1].bar(bin_edges_x[:-1]+shift1, hist_x, color='r', alpha=0.9, width=bin_width)
    ax[1].bar(bin_edges_y[:-1]+shift2, hist_y, color='g', alpha=0.9, width=bin_width)
    ax[1].bar(bin_edges_z[:-1]+shift3, hist_z, color='b', alpha=0.9, width=bin_width)
    # ax[1].legend([f'Ground truth\nmean roundness = {gt_mean}', f'Automatic\nmean roundness = {my_mean}', f'Semiautomatic\nmean roundness = {s_mean}'], fontsize='small')
    # Add title and axis labels
    ax[1].set_title('Whole dataset', fontsize=22)
    ax[1].set_xlabel('Roundness of fitted ellipses', fontsize=16)
    ax[1].set_ylabel('Probability', fontsize=16)
    ax[1].legend(['Ground truth', 'Automatic', 'Semiautomatic'], fontsize='medium')
    ax[1].set_xlim(min_roundness-0.02, 1.02)
    ax[1].tick_params(axis='x', labelsize=12)
    ax[1].tick_params(axis='y', labelsize=12)
    if save_plots:
        plt.savefig(os.path.join(plot_path, 'whole.svg'), format='svg')
    plt.show()

    # Correlation for the number of found ellipses
    num_all_my_ellipses = np.array(num_all_my_ellipses)
    num_all_gt_ellipses = np.array(num_all_gt_ellipses)
    num_all_s_ellipses =  np.array(num_all_s_ellipses)

    sorted_indices = np.argsort(num_all_gt_ellipses)
    num_all_gt_ellipses = num_all_gt_ellipses[sorted_indices]
    num_all_my_ellipses = num_all_my_ellipses[sorted_indices]
    num_all_s_ellipses = num_all_s_ellipses[sorted_indices]

    my_corr_coeff = np.corrcoef(num_all_gt_ellipses, num_all_my_ellipses)[0, 1]
    my_r_squared = round(my_corr_coeff ** 2, 2)
    my_r_squared ='{:.2f}'.format(my_r_squared)

    s_corr_coeff = np.corrcoef(num_all_gt_ellipses, num_all_s_ellipses)[0, 1]
    s_r_squared = round(s_corr_coeff ** 2, 2)
    s_r_squared ='{:.2f}'.format(s_r_squared)

    # Fit a linear function to the data
    my_coefficients = np.polyfit(num_all_gt_ellipses, num_all_my_ellipses, 1)
    my_fit_line = np.poly1d(my_coefficients)

    # Fit a linear function to the data
    s_coefficients = np.polyfit(num_all_gt_ellipses, num_all_s_ellipses, 1)
    s_fit_line = np.poly1d(s_coefficients)


    s_fitted_values = np.polyval(s_coefficients, num_all_gt_ellipses)
    s_residuals = num_all_s_ellipses - s_fitted_values
    s_residuals_sd = np.std(s_residuals)

    my_fitted_values = np.polyval(my_coefficients, num_all_gt_ellipses)
    my_residuals = num_all_my_ellipses - my_fitted_values
    my_residuals_sd = np.std(my_residuals)


    s_derivative_coefficients = np.polyder(s_coefficients)
    my_derivative_coefficients = np.polyder(my_coefficients)

    s_tg = np.polyval(s_derivative_coefficients, 1000)
    my_tg = np.polyval(my_derivative_coefficients, 1000)
    s_tg = '{:.2f}'.format(round(s_tg, 2))
    my_tg = '{:.2f}'.format(round(my_tg, 2))
    s_residuals_sd = round(s_residuals_sd)
    my_residuals_sd = round(my_residuals_sd)


    print(f's tangent {s_tg}+-{s_residuals_sd}')
    print(f'my tangent {my_tg}+-{my_residuals_sd}')


    fig, ax = plt.subplots(1, 1, figsize=(7, 7))
    # ax = ax.flatten()
    sns.set_theme()
    ax.plot(num_all_gt_ellipses, my_fit_line(num_all_gt_ellipses), color='g', linestyle='--', alpha=0.6, label=f'Automatic linear fit: s = {my_tg}±{my_residuals_sd}')
    ax.plot(num_all_gt_ellipses, s_fit_line(num_all_gt_ellipses), color='b', linestyle='--', alpha=0.6, label=f'Semiautomatic linear fit: s = {s_tg}±{s_residuals_sd}')
    ax.scatter(num_all_gt_ellipses, num_all_my_ellipses, color='g', marker='o', label=f'Automatic: R² = {my_r_squared}', alpha=np.ones_like(num_all_gt_ellipses))
    ax.scatter(num_all_gt_ellipses, num_all_s_ellipses, color='b', marker='s', label=f'Semiautomatic: R² = {s_r_squared}', alpha=np.ones_like(num_all_gt_ellipses))

    ax.legend(fontsize='medium')
    ax.set_xlabel('Ground truth number of fenestrations per image', fontsize=16)
    ax.set_ylabel('Found number of fenestrations per image', fontsize=16)
    ax.set_title(f'Correlation plot of the number of found fenestrations\nwith the automatic and semiautomatic method', fontsize=22)
    ax.grid(True)
    ax.tick_params(axis='x', labelsize=12)
    ax.tick_params(axis='y', labelsize=12)
    if save_plots:
        plt.savefig(os.path.join(plot_path, 'corr_num.svg'), format='svg')
    plt.show()



    print('num ellipses')
    print(f'gt {np.sum(num_all_gt_ellipses)}, my {np.sum(num_all_my_ellipses)}, s {np.sum(num_all_s_ellipses)}')
    print(my_dice_scores)
    print(s_dice_scores)
    print('my')
    for dice in my_dice_scores:
        print(round(dice, 2))
    my_dice_scores = np.array(my_dice_scores)
    my_mean_dice = round(np.mean(my_dice_scores), 2)
    my_std_dice = round(np.std(my_dice_scores), 2)
    print(f'Auto mean dice: {my_mean_dice} +- {my_std_dice}')
    print('s')
    for dice in s_dice_scores:
        print(round(dice, 2))
    s_dice_scores = np.array(s_dice_scores)
    s_mean_dice = round(np.mean(s_dice_scores), 2)
    s_std_dice = round(np.std(s_dice_scores), 2)
    print(f'Semiauto mean dice: {s_mean_dice} +- {s_std_dice}')

    # correlation of means


    all_my_means = np.array(all_my_means)
    all_gt_means = np.array(all_gt_means)
    all_s_means =  np.array(all_s_means)

    sorted_indices = np.argsort(all_gt_means)
    all_gt_means = all_gt_means[sorted_indices]
    all_my_means = all_my_means[sorted_indices]
    all_s_means = all_s_means[sorted_indices]

    my_corr_coeff = np.corrcoef(all_gt_means, all_my_means)[0, 1]
    my_r_squared = round(my_corr_coeff ** 2, 2)
    my_r_squared ='{:.2f}'.format(my_r_squared)

    s_corr_coeff = np.corrcoef(all_gt_means, all_s_means)[0, 1]
    s_r_squared = round(s_corr_coeff ** 2, 2)
    s_r_squared ='{:.2f}'.format(s_r_squared)

    # Fit a linear function to the data
    my_coefficients = np.polyfit(all_gt_means, all_my_means, 1)
    my_fit_line = np.poly1d(my_coefficients)

    # Fit a linear function to the data
    s_coefficients = np.polyfit(all_gt_means, all_s_means, 1)
    s_fit_line = np.poly1d(s_coefficients)

    s_fitted_values = np.polyval(s_coefficients, all_gt_means)
    s_residuals = all_s_means - s_fitted_values
    s_residuals_sd = np.std(s_residuals)

    my_fitted_values = np.polyval(my_coefficients, all_gt_means)
    my_residuals = all_my_means - my_fitted_values
    my_residuals_sd = np.std(my_residuals)


    s_derivative_coefficients = np.polyder(s_coefficients)
    my_derivative_coefficients = np.polyder(my_coefficients)

    s_tg = np.polyval(s_derivative_coefficients, 150)
    my_tg = np.polyval(my_derivative_coefficients, 150)
    s_tg = '{:.2f}'.format(round(s_tg, 2))
    my_tg = '{:.2f}'.format(round(my_tg, 2))
    s_residuals_sd = round(s_residuals_sd)
    my_residuals_sd = round(my_residuals_sd)

    print('mean values')
    print(f's tangent {s_tg}+-{s_residuals_sd}')
    print(f'my tangent {my_tg}+-{my_residuals_sd}')

    fig, ax = plt.subplots(1, 1, figsize=(7, 7))
    sns.set_theme()
    ax.plot(all_gt_means, my_fit_line(all_gt_means), color='g', linestyle='--', alpha=0.6, label=f'Automatic linear fit: s = {my_tg}±{my_residuals_sd}')
    ax.plot(all_gt_means, s_fit_line(all_gt_means), color='b', linestyle='--', alpha=0.6, label=f'Semiautomatic linear fit: s = {s_tg}±{s_residuals_sd}')
    ax.scatter(all_gt_means, all_my_means, color='g', marker='o', label=f'Automatic: R² = {my_r_squared}', alpha=np.ones_like(all_gt_means))
    ax.scatter(all_gt_means, all_s_means, color='b', marker='s', label=f'Semiautomatic: R² = {s_r_squared}', alpha=np.ones_like(all_gt_means))

    ax.legend(fontsize='medium')
    ax.set_xlabel('Ground truth mean fenestration\nequivalent diameter per image (nm)', fontsize=16)
    ax.set_ylabel('Found mean fenestration\nequivalent diameter per image (nm)', fontsize=16)
    ax.set_title(f'Correlation plot of the mean fenestration diameter\nwith the automatic and semiautomatic method', fontsize=22)
    ax.grid(True)
    ax.tick_params(axis='x', labelsize=12)
    ax.tick_params(axis='y', labelsize=12)
    x_ticks = list(range(110, 180+1, 10))
    ax.set_xticks(x_ticks)
    ax.set_yticks(x_ticks)
    if save_plots:
        plt.savefig(os.path.join(plot_path, 'corr_diameter.svg'), format='svg')
    plt.show()
    print('gt means')
    for mean in all_gt_means:
        print(mean)
    print('my means')
    for mean in all_my_means:
        print(mean)
    print('s means')
    for mean in all_s_means:
        print(mean)



    print('porosity')
    print('gt')
    for p in all_gt_porosities_pix:
        print(p)
    print('s')
    for p in all_s_porosities_pix:
        print(p)
    print('my')
    for p in all_my_porosities_pix:
        print(p)

    # Porosity correlation
    all_gt_porosities_pix = np.array(all_gt_porosities_pix)
    all_s_porosities_pix = np.array(all_s_porosities_pix)
    all_my_porosities_pix =  np.array(all_my_porosities_pix)

    sorted_indices = np.argsort(all_gt_porosities_pix)
    all_gt_porosities_pix = all_gt_porosities_pix[sorted_indices]
    all_s_porosities_pix = all_s_porosities_pix[sorted_indices]
    all_my_porosities_pix = all_my_porosities_pix[sorted_indices]

    my_corr_coeff = np.corrcoef(all_gt_porosities_pix, all_my_porosities_pix)[0, 1]
    my_r_squared = round(my_corr_coeff ** 2, 2)
    my_r_squared ='{:.2f}'.format(my_r_squared)

    s_corr_coeff = np.corrcoef(all_gt_porosities_pix, all_s_porosities_pix)[0, 1]
    s_r_squared = round(s_corr_coeff ** 2, 2)
    s_r_squared ='{:.2f}'.format(s_r_squared)

    # Fit a linear function to the data
    my_coefficients = np.polyfit(all_gt_porosities_pix, all_my_porosities_pix, 1)
    my_fit_line = np.poly1d(my_coefficients)

    # Fit a linear function to the data
    s_coefficients = np.polyfit(all_gt_porosities_pix, all_s_porosities_pix, 1)
    s_fit_line = np.poly1d(s_coefficients)

    s_fitted_values = np.polyval(s_coefficients, all_gt_porosities_pix)
    s_residuals = all_s_porosities_pix - s_fitted_values
    s_residuals_sd = np.std(s_residuals)

    my_fitted_values = np.polyval(my_coefficients, all_gt_porosities_pix)
    my_residuals = all_my_porosities_pix - my_fitted_values
    my_residuals_sd = np.std(my_residuals)


    s_derivative_coefficients = np.polyder(s_coefficients)
    my_derivative_coefficients = np.polyder(my_coefficients)

    s_tg = np.polyval(s_derivative_coefficients, 5)
    my_tg = np.polyval(my_derivative_coefficients, 5)
    s_tg = '{:.2f}'.format(round(s_tg, 2))
    my_tg = '{:.2f}'.format(round(my_tg, 2))
    s_residuals_sd = round(s_residuals_sd, 2)
    my_residuals_sd = round(my_residuals_sd, 2)

    # print('mean values')
    # print(f's tangent {s_tg}+-{s_residuals_sd}')
    # print(f'my tangent {my_tg}+-{my_residuals_sd}')

    fig, ax = plt.subplots(1, 1, figsize=(7, 7))
    sns.set_theme()
    ax.plot(all_gt_porosities_pix, my_fit_line(all_gt_porosities_pix), color='g', linestyle='--', alpha=0.6, label=f'Automatic linear fit: s = {my_tg}±{my_residuals_sd}')
    ax.plot(all_gt_porosities_pix, s_fit_line(all_gt_porosities_pix), color='b', linestyle='--', alpha=0.6, label=f'Semiautomatic linear fit: s = {s_tg}±{s_residuals_sd}')
    ax.scatter(all_gt_porosities_pix, all_my_porosities_pix, color='g', marker='o', label=f'Automatic: R² = {my_r_squared}', alpha=np.ones_like(all_gt_porosities_pix))
    ax.scatter(all_gt_porosities_pix, all_s_porosities_pix, color='b', marker='s', label=f'Semiautomatic: R² = {s_r_squared}', alpha=np.ones_like(all_gt_porosities_pix))

    ax.legend(fontsize='medium')
    ax.set_xlabel('Ground truth porosity per image (%)', fontsize=16)
    ax.set_ylabel('Found porosity per image (%)', fontsize=16)
    ax.set_title(f'Correlation plot of cell porosity of fenestrations detected\nwith the automatic and semiautomatic method', fontsize=22)
    ax.grid(True)
    ax.tick_params(axis='x', labelsize=12)
    ax.tick_params(axis='y', labelsize=12)
    # x_ticks = list(range(110, 180+1, 10))
    # ax.set_xticks(x_ticks)
    # ax.set_yticks(x_ticks)
    if save_plots:
        plt.savefig(os.path.join(plot_path, 'corr_porosity_pix.svg'), format='svg')
    plt.show()



    print('porosity')
    print('gt')
    for p in all_gt_porosities_ell:
        print(p)
    print('s')
    for p in all_s_porosities_ell:
        print(p)
    print('my')
    for p in all_my_porosities_ell:
        print(p)

    # Porosity correlation
    all_gt_porosities_ell = np.array(all_gt_porosities_ell)
    all_s_porosities_ell = np.array(all_s_porosities_ell)
    all_my_porosities_ell =  np.array(all_my_porosities_ell)

    sorted_indices = np.argsort(all_gt_porosities_ell)
    all_gt_porosities_ell = all_gt_porosities_ell[sorted_indices]
    all_s_porosities_ell = all_s_porosities_ell[sorted_indices]
    all_my_porosities_ell = all_my_porosities_ell[sorted_indices]

    my_corr_coeff = np.corrcoef(all_gt_porosities_ell, all_my_porosities_ell)[0, 1]
    my_r_squared = round(my_corr_coeff ** 2, 2)
    my_r_squared ='{:.2f}'.format(my_r_squared)

    s_corr_coeff = np.corrcoef(all_gt_porosities_ell, all_s_porosities_ell)[0, 1]
    s_r_squared = round(s_corr_coeff ** 2, 2)
    s_r_squared ='{:.2f}'.format(s_r_squared)

    # Fit a linear function to the data
    my_coefficients = np.polyfit(all_gt_porosities_ell, all_my_porosities_ell, 1)
    my_fit_line = np.poly1d(my_coefficients)

    # Fit a linear function to the data
    s_coefficients = np.polyfit(all_gt_porosities_ell, all_s_porosities_ell, 1)
    s_fit_line = np.poly1d(s_coefficients)

    s_fitted_values = np.polyval(s_coefficients, all_gt_porosities_ell)
    s_residuals = all_s_porosities_ell - s_fitted_values
    s_residuals_sd = np.std(s_residuals)

    my_fitted_values = np.polyval(my_coefficients, all_gt_porosities_ell)
    my_residuals = all_my_porosities_ell - my_fitted_values
    my_residuals_sd = np.std(my_residuals)


    s_derivative_coefficients = np.polyder(s_coefficients)
    my_derivative_coefficients = np.polyder(my_coefficients)

    s_tg = np.polyval(s_derivative_coefficients, 5)
    my_tg = np.polyval(my_derivative_coefficients, 5)
    s_tg = '{:.2f}'.format(round(s_tg, 2))
    my_tg = '{:.2f}'.format(round(my_tg, 2))
    s_residuals_sd = round(s_residuals_sd, 2)
    my_residuals_sd = round(my_residuals_sd, 2)

    # print('mean values')
    # print(f's tangent {s_tg}+-{s_residuals_sd}')
    # print(f'my tangent {my_tg}+-{my_residuals_sd}')

    fig, ax = plt.subplots(1, 1, figsize=(7, 7))
    sns.set_theme()
    ax.plot(all_gt_porosities_ell, my_fit_line(all_gt_porosities_ell), color='g', linestyle='--', alpha=0.6, label=f'Automatic linear fit: s = {my_tg}±{my_residuals_sd}')
    ax.plot(all_gt_porosities_ell, s_fit_line(all_gt_porosities_ell), color='b', linestyle='--', alpha=0.6, label=f'Semiautomatic linear fit: s = {s_tg}±{s_residuals_sd}')
    ax.scatter(all_gt_porosities_ell, all_my_porosities_ell, color='g', marker='o', label=f'Automatic: R² = {my_r_squared}', alpha=np.ones_like(all_gt_porosities_ell))
    ax.scatter(all_gt_porosities_ell, all_s_porosities_ell, color='b', marker='s', label=f'Semiautomatic: R² = {s_r_squared}', alpha=np.ones_like(all_gt_porosities_ell))

    ax.legend(fontsize='medium')
    ax.set_xlabel('Ground truth porosity per image (%)', fontsize=16)
    ax.set_ylabel('Found porosity per image (%)', fontsize=16)
    ax.set_title(f'Correlation plot of cell porosity computed with fitted ellipses\nwith the automatic and semiautomatic method', fontsize=22)
    ax.grid(True)
    ax.tick_params(axis='x', labelsize=12)
    ax.tick_params(axis='y', labelsize=12)
    # x_ticks = list(range(110, 180+1, 10))
    # ax.set_xticks(x_ticks)
    # ax.set_yticks(x_ticks)
    if save_plots:
        plt.savefig(os.path.join(plot_path, 'corr_porosity_ell.svg'), format='svg')
    plt.show()


# fen freq
    print('frequency')
    print('gt')
    for p in all_gt_frequencies:
        print(p)
    print('s')
    for p in all_s_frequencies:
        print(p)
    print('my')
    for p in all_my_frequencies:
        print(p)

    # Porosity correlation
    all_gt_frequencies = np.array(all_gt_frequencies)
    all_s_frequencies = np.array(all_s_frequencies)
    all_my_frequencies =  np.array(all_my_frequencies)

    sorted_indices = np.argsort(all_gt_frequencies)
    all_gt_frequencies = all_gt_frequencies[sorted_indices]
    all_s_frequencies = all_s_frequencies[sorted_indices]
    all_my_frequencies = all_my_frequencies[sorted_indices]

    my_corr_coeff = np.corrcoef(all_gt_frequencies, all_my_frequencies)[0, 1]
    my_r_squared = round(my_corr_coeff ** 2, 2)
    my_r_squared ='{:.2f}'.format(my_r_squared)

    s_corr_coeff = np.corrcoef(all_gt_frequencies, all_s_frequencies)[0, 1]
    s_r_squared = round(s_corr_coeff ** 2, 2)
    s_r_squared ='{:.2f}'.format(s_r_squared)

    # Fit a linear function to the data
    my_coefficients = np.polyfit(all_gt_frequencies, all_my_frequencies, 1)
    my_fit_line = np.poly1d(my_coefficients)

    # Fit a linear function to the data
    s_coefficients = np.polyfit(all_gt_frequencies, all_s_frequencies, 1)
    s_fit_line = np.poly1d(s_coefficients)

    s_fitted_values = np.polyval(s_coefficients, all_gt_frequencies)
    s_residuals = all_s_frequencies - s_fitted_values
    s_residuals_sd = np.std(s_residuals)

    my_fitted_values = np.polyval(my_coefficients, all_gt_frequencies)
    my_residuals = all_my_frequencies - my_fitted_values
    my_residuals_sd = np.std(my_residuals)


    s_derivative_coefficients = np.polyder(s_coefficients)
    my_derivative_coefficients = np.polyder(my_coefficients)

    s_tg = np.polyval(s_derivative_coefficients, 5)
    my_tg = np.polyval(my_derivative_coefficients, 5)
    s_tg = '{:.2f}'.format(round(s_tg, 2))
    my_tg = '{:.2f}'.format(round(my_tg, 2))
    s_residuals_sd = round(s_residuals_sd, 2)
    my_residuals_sd = round(my_residuals_sd, 2)


    fig, ax = plt.subplots(1, 1, figsize=(7, 7))
    sns.set_theme()
    ax.plot(all_gt_frequencies, my_fit_line(all_gt_frequencies), color='g', linestyle='--', alpha=0.6, label=f'Automatic linear fit: s = {my_tg}±{my_residuals_sd}')
    ax.plot(all_gt_frequencies, s_fit_line(all_gt_frequencies), color='b', linestyle='--', alpha=0.6, label=f'Semiautomatic linear fit: s = {s_tg}±{s_residuals_sd}')
    ax.scatter(all_gt_frequencies, all_my_frequencies, color='g', marker='o', label=f'Automatic: R² = {my_r_squared}', alpha=np.ones_like(all_gt_frequencies))
    ax.scatter(all_gt_frequencies, all_s_frequencies, color='b', marker='s', label=f'Semiautomatic: R² = {s_r_squared}', alpha=np.ones_like(all_gt_frequencies))

    ax.legend(fontsize='medium')
    ax.set_xlabel('Fenestration frequency per image\n(fenestrations/μm²)', fontsize=16)
    ax.set_ylabel('Detected fenestration frequency per image\n(fenestrations/μm²)', fontsize=16)
    ax.set_title(f'Correlation plot of fenestration frequency detected\nwith the automatic and semiautomatic method', fontsize=22)
    ax.grid(True)
    ax.tick_params(axis='x', labelsize=12)
    ax.tick_params(axis='y', labelsize=12)
    # x_ticks = list(range(110, 180+1, 10))
    # ax.set_xticks(x_ticks)
    # ax.set_yticks(x_ticks)
    if save_plots:
        plt.savefig(os.path.join(plot_path, 'corr_freq.svg'), format='svg')
    plt.show()




    #     # if remove_false_fenestrations:
    #     #     new_mask_filt = remove_fenestrations(new_mask, min_diameter_nm, max_diameter_nm, min_roundness, pixel_size_nm)
    #     #     # current_dice_score_filt = compute_dice_score(ground_truth_mask, new_mask_filt)
    #     #     # dice_scores_filt.append(current_dice_score_filt)
    #     #     # print(f'Image Dice score: {round(current_dice_score*100, 1)}, ({round(current_dice_score_filt*100, 1)})')
    #     #     # file.write(f'Image Dice score: {round(current_dice_score*100, 1)}, ({round(current_dice_score_filt*100, 1)})\n')
    #     # else:
    #     #     # print(f'Image Dice score: {round(current_dice_score*100, 1)}')
    #     #     # file.write(f'Image Dice score: {round(current_dice_score*100, 1)}\n')

    # # dice_scores = np.array(dice_scores)
    # # mean_dice = round(np.mean(dice_scores)*100, 1)
    # # std_dice = round(np.std(dice_scores)*100, 1)
    # # if remove_false_fenestrations:
    # #     dice_scores_filt = np.array(dice_scores_filt)
    # #     mean_dice_filt = round(np.mean(dice_scores_filt)*100, 1)
    # #     std_dice_filt = round(np.std(dice_scores_filt)*100, 1)
    # #     print(f'{model_name} Mean dice: {mean_dice} +- {std_dice} ({mean_dice_filt} +- {std_dice_filt})\n')
    # #     file.write(f'{model_name} Mean dice: {mean_dice} += {std_dice} ({mean_dice_filt} +- {std_dice_filt})\n\n')
    # # else:
    # #     print(f'{model_name} Mean dice: {mean_dice} +- {std_dice}\n')
    # #     file.write(f'{model_name} Mean dice: {mean_dice} += {std_dice}\n\n')





In [ ]:
model = build_model('vgg11+none', 0.0, 'dice+bce')
loaded_state_dict = torch.load('./gdrive/MyDrive/lsecs/vgg11+imagenet_dice+bce_med5.pth')
model.load_state_dict(loaded_state_dict)
model.eval()
square_section = cv.imread('./gdrive/MyDrive/lsecs/cropped_selections/patches_med5/val_image_patches/II_Y10_04_3_patch_1_0.tif', cv.IMREAD_GRAYSCALE)
square_tensor = test_transform(image=square_section)['image'].unsqueeze(0).to(DEVICE)  # Add batch  dimension


with torch.no_grad():
    output = torch.sigmoid(model(square_tensor))
    # output = (output > 0.5).float()
    output = output.cpu().data.numpy().squeeze(0).squeeze()
    output = output*255
    cv2_imshow(square_section)
    cv2_imshow(output)

    # # # Forward pass through the model
    # # with torch.no_grad():
    # #     output = torch.sigmoid(model(square_tensor)).float()

    # # Scale the probablity to 0-255

    # # output = output.to(torch.uint8)
    # # output_pil = output.squeeze(0).cpu().numpy().squeeze()
    # # cv2_imshow(output_pil)
    # output_probs[x:x+window_size, y:y+window_size] += output*weighting_window
    # # Crop
    # # cv.imwrite(os.path.join(output_folder, "probs"+".png"), output_probs)

    # output_probs = output_probs[oh:original_height+oh, ow:original_width+ow]
    # weights *= 255
    # # weights = weights[:original_height, :original_width]*255
    # # tryout = tryout[:original_height, :original_width]*255

    # # Apply weights
    # # output_probs /= weights

    # # Create image from mask
    # # output_mask = np.where(output_probs > 127, 255, 0)
    # output_mask = output_mask.astype(np.uint8)
    # return output_mask

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define paths to your images
image_paths = ['./gdrive/MyDrive/lsecs/dice_score_test/lc/2022-11-10_Pill18_C0_1h_M1_18.tif', './gdrive/MyDrive/lsecs/clahe/p2.tif']







# Read and normalize images
images = []
normalized_images = []
for path in image_paths:
    # Read image with OpenCV in grayscale
    img = cv.imread(path, cv.IMREAD_GRAYSCALE)
    # Convert image to float32
    print(np.mean(img))
    images.append(img)
    img = normalize_hist(img)
    # clahe = cv.createCLAHE(clipLimit=np.mean(img)/10, tileGridSize=(8, 8))
    # img = clahe.apply(img)
    # img = cv.medianBlur(img, 3)
    img = img.astype('float32')
    img = (img - 0.5 * 255.0) / (0.5 * 255.0)
    print(np.mean(img))
    # Normalize image to have mean 0 and std 1
    # img /= 255.0  # Scale to range [0, 1]
    # img -= np.mean(img)  # Subtract mean
    # img /= np.std(img)   # Divide by std
    # Append normalized image to the list



    normalized_images.append(img)

# Plot the images

fig, axes = plt.subplots(2, 4, figsize=(18, 8))

for i in range(2):
    # Plot original image
    # axes[i, 0].imshow(images[i], cmap='gray')
    # axes[i, 0].set_title('Original Image')
    # axes[i, 0].axis('off')

    # # Plot normalized image
    # axes[i, 1].imshow(normalized_images[i], cmap='gray', vmin=-1, vmax=1)
    # axes[i, 1].set_title('Normalized Image')
    # axes[i, 1].axis('off')

    axes[i, 2].hist(images[i].flatten(), bins=50, color='blue', alpha=0.5)
    axes[i, 2].set_ylabel('Frequency', fontsize='medium')
    axes[i, 2].set_xlabel('Intensity value', fontsize='medium')

    axes[i, 3].hist(normalized_images[i].flatten(), bins=50, color='blue', alpha=0.5)
    axes[i, 3].set_ylabel('Frequency', fontsize='medium')
    axes[i, 3].set_xlabel('Intensity value', fontsize='medium')
# plot_path = './gdrive/MyDrive/lsecs/plots/'
# plt.savefig(os.path.join(plot_path, 'hists.svg'), format='svg')
plt.tight_layout()
plt.show()


# for i, ax in enumerate(axes.flat):
#     ax.imshow(images[i], cmap='gray', vmin=-1, vmax=1)
#     ax.axis('off')

# plt.show()

# Bioimageio stuff

In [ ]:
# !pip install "bioimageio.core>=0.5,<0.6"

In [ ]:
# @torch.jit.ignore
# def call_np(tensor) -> torch.Tensor:
#   na = tensor.numpy()
#   # Interesting stuff here
#   tt = torch.tensor(na)
#   return tt

# class MyModule(nn.Module):
#     @torch.jit.export
#     def forward(self, tensor):
#         done = call_np(tensor)
#         print (done)

# scripted_module = torch.jit.script(MyModule())
# print(scripted_module.forward.graph)
# empty_tensor = torch.empty(3, 4)
# scripted_module.forward(empty_tensor)










In [ ]:
# import torchvision.transforms as transforms
# import numpy as np

# @torch.jit.ignore
# def denoise_image(tensor) -> torch.Tensor:
#   na = tensor.numpy()
#   # Interesting stuff here
#   tt = torch.tensor(na)
#   return tt

# class FunctionWrapper(nn.Module):
#   def __init__(self, model):
#     super(FunctionWrapper, self).__init__()
#     self.model = model

#     @torch.jit.export
#     def forward(self, tensor):
#         denoised = denoise_image(tensor)
#         return self.model(denoised)



# device = torch.device('cpu')
# model = UNET(in_channels=1, out_channels=1, device='cpu')
# model.load_state_dict(torch.load(biomodel_path, map_location=device))
# # model.to(device=device)
# model = torch.jit.script(model)
# # wrapper = FunctionWrapper(model)
# wrapper.to(device=device)
# # wrapper = PreprocessingWrapper(denoise, model)
# # model = torch.jit.script(wrapper)
# #
# model.eval()
# torchscript_weights_path = os.path.join(biomodel_folder, 'torchscript_weights.pt')
# torch.jit.save(model, torchscript_weights_path)

# preprocessing=[[{"name": "scale_range",
#                  "kwargs": {"axes": "xy",
#                           #  "min_percentile": min_percentile,
#                             # "max_percentile": max_percentile,
#                             "mode": "per_sample"
#                             }}]]

# threshold = 0.5
# postprocessing = [[{"name": "binarize", "kwargs": {"threshold": threshold}}]]


In [ ]:
# input = np.random.rand(1, 1, 512, 512).astype("float32")  # an example input
# test_inputs = os.path.join(biomodel_folder, "test-input.npy")
# test_outputs = os.path.join(biomodel_folder, "test-output.npy")
# np.save(test_inputs, input)
# with torch.no_grad():
#   output = model(torch.from_numpy(input)).cpu().numpy() # copy to cpu(is on gpu because of jit.script)
#   output = output > threshold
# np.save(test_outputs, output)

# print(input.shape)
# print(output.shape)

In [ ]:
# # create markdown documentation for your model
# # this should describe how the model was trained, (and on which data)
# # and also what to take into consideration when running the model, especially how to validate the model
# # here, we just create a stub documentation
# doc_path = os.path.join(biomodel_folder, "doc.md")
# with open(doc_path, "w") as f:
#     f.write("# My First Model\n")

In [ ]:
# from bioimageio.core.build_spec import build_model
# import torch
# # now we can use the build_model function to create the zipped package.
# # it takes the path to the weights and data we have just created, as well as additional information
# # that will be used to add metadata to the rdf.yaml file in the model zip
# # we only use a subset of the available options here, please refer to the advanced examples and to the
# # function signature of build_model in order to get an overview of the full functionality
# build_model(
#     # the weight file and the type of the weights
#     weight_uri= torchscript_weights_path,
#     weight_type="torchscript",
#     # the test input and output data as well as the description of the tensors
#     # these are passed as list because we support multiple inputs / outputs per model
#     test_inputs=[test_inputs],
#     test_outputs=[test_outputs],
#     input_axes=["bcyx"],
#     output_axes=["bcyx"],
#     # where to save the model zip, how to call the model and a short description of it
#     output_path=os.path.join(biomodel_folder,"model.zip"),
#     name="MyFirstModel",
#     description="a fancy new model",
#     # additional metadata about authors, licenses, citation etc.
#     authors=[{"name": "Gizmo"}],
#     license="CC-BY-4.0",
#     documentation=doc_path,
#     tags=["nucleus-segmentation"],  # the tags are used to make models more findable on the website
#     cite=[{"text": "Gizmo et al.", "doi": "10.1002/xyzacab123"}],
#     pytorch_version=torch.__version__,
#     preprocessing=preprocessing,
#     postprocessing=postprocessing
# )

In [ ]:
# # finally, we test that the expected outptus are reproduced when running the model.
# # the 'test_model' function runs this test.
# # it will output a list of dictionaries. each dict gives the status of a different test that is being run
# # if all of them contain "status": "passed" then all tests were successful
# from bioimageio.core.resource_tests import test_model
# import bioimageio.core
# my_model = bioimageio.core.load_resource_description(os.path.join(biomodel_folder,"model.zip"))
# test_model(my_model)